In [40]:
import numpy as np
from scipy.stats import pearsonr
from dtaidistance import dtw
import networkx as nx
from gensim.models import Word2Vec
from sklearn.cluster import KMeans
import pandas as pd

def strict_image_method(data, labels, window_size=5):
    """
    完全按照图片描述的三个步骤实现：
    1. 构建图结构（节点为时间窗口,边由两个条件：Pearson相关系数和DTW距离确定）
    2. 基于随机游走和Word2Vec实现DeepWalk嵌入
    3. 利用KMeans聚类并通过聚类较小的那一类判断异常窗口,再转换回原始时间点,
       并统计每个传感器的异常频率（使用Z-score计算）。
    """
    # 1. 构建图结构：节点为连续时间窗口
    G = nx.Graph()
    n_windows = len(data) - window_size + 1
    for i in range(n_windows):
        window = data[i:i+window_size]
        G.add_node(i, values=window)
    
    # 添加边（严格按两个条件判断）
    for i in range(n_windows):
        for j in range(i+1, min(i+20, n_windows)):  # 限制连接范围
            # 条件1：Pearson相关系数
            corr = pearsonr(G.nodes[i]['values'].flatten(), 
                            G.nodes[j]['values'].flatten())[0]
            # 条件2：DTW距离（这里先计算各窗口沿时间方向的均值序列）
            dtw_dist = dtw.distance(G.nodes[i]['values'].mean(axis=1),
                                    G.nodes[j]['values'].mean(axis=1))
            
            if abs(corr) > 0.6 and dtw_dist < 1.5:  # 严格阈值条件
                G.add_edge(i, j, weight=abs(corr))
    
    # 2. DeepWalk：利用随机游走获得节点序列,再用Word2Vec训练嵌入
    def random_walk(G, walk_length=15):
        walks = []
        for _ in range(10):  # 每个节点执行10次随机游走
            for node in G.nodes():
                walk = [str(node)]
                while len(walk) < walk_length:
                    current = int(walk[-1])
                    neighbors = list(G.neighbors(current))
                    if neighbors:
                        walk.append(str(np.random.choice(neighbors)))
                    else:
                        break
                walks.append(walk)
        return walks

    walks = random_walk(G)
    model = Word2Vec(walks, vector_size=32, window=5, min_count=0, sg=1)
    embeddings = np.array([model.wv[str(i)] for i in G.nodes()])
    
    # 3. 异常检测：使用KMeans聚类,并认为聚类数量较少的一类对应异常窗口
    kmeans = KMeans(n_clusters=2, random_state=42)
    clusters = kmeans.fit_predict(embeddings)
    # 选择窗口数目较少的聚类作为异常窗口
    anomalies = np.where(clusters == np.argmin(np.bincount(clusters)))[0]
    
    # 将异常窗口映射回原始时间点
    anomaly_windows = [i for i in anomalies if i in G.nodes()]
    anomaly_points = []
    for win in anomaly_windows:
        anomaly_points.extend(range(win, win+window_size))
    
    # 统计每个传感器的异常频率（仅统计标签为1的真实异常点）
    sensor_scores = np.zeros(data.shape[1])
    for point in set(anomaly_points):
        if point < len(labels) and labels[point] == 1:
            for sensor in range(data.shape[1]):
                # 计算局部Z-score
                mean_val = np.mean(data[max(0, point-100):point+1, sensor])
                std_val = np.std(data[max(0, point-100):point+1, sensor])
                sensor_scores[sensor] += abs(data[point, sensor] - mean_val) / (std_val + 1e-8)
    
    # 这里简单返回异常得分大于0的传感器（您可根据需求设定更复杂的阈值）
    return np.where(sensor_scores > 0)[0]

# 循环遍历 /data/processed/ 下的所有测试数据文件以omi-开头的文件
def get_test_files():
    import os
    files = os.listdir('data/processed/')
    test_files = [f for f in files if f.startswith('omi-') and f.endswith('_test.pkl')]
    return test_files



if __name__ == "__main__":


    # 加载测试数据与标签
    test_data = pd.read_pickle('data/processed/omi-1_test.pkl')
    test_labels = pd.read_pickle('data/processed/omi-1_test_label.pkl')

    # 如果读取结果为numpy数组,则转换为DataFrame
    if isinstance(test_data, np.ndarray):
        test_data = pd.DataFrame(test_data)
    if isinstance(test_labels, np.ndarray):
        test_labels = pd.DataFrame(test_labels, columns=["label"])
    
    # 设定需要检测的时段
    segments = [(760, 765), (1064, 1298), (2758, 2772)]
    results = {}
    for segment in segments:
        # 提取指定时段数据和对应标签（转换为numpy数组）
        seg_data = test_data.iloc[segment[0]:segment[1]+1].to_numpy()
        seg_labels = test_labels.iloc[segment[0]:segment[1]+1]["label"].to_numpy()
        anomalies = strict_image_method(seg_data, seg_labels)
        results[f"{segment[0]}-{segment[1]}"] = anomalies
        print(f"时段 {segment[0]}-{segment[1]} 检测到的异常传感器：{sorted(anomalies)}")
    
    # 结果对比（示例）：与预期结果对比
    expected = {
        "760-765": [4,5,7,8,9,10,11,12,13,14,15,16,17,18,19],
        "1064-1298": [6,7,9,10,11,12,13,14,15,16,17,18,19],
        "2758-2772": [5,6,7,8,9,10,11,12,13,14,15,16,17,18,19]
    }
    
    for seg in results:
        detected = set(results[seg])
        actual = set(expected[seg])
        print(f"时段 {seg}：")
        print(f"→ 正确检测数：{len(detected & actual)}/{len(actual)}")
        print(f"→ 漏检传感器：{sorted(actual - detected)}")
        print(f"→ 误检传感器：{sorted(detected - actual)}")
        print("="*50)


时段 760-765 检测到的异常传感器：[0, 3, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
时段 1064-1298 检测到的异常传感器：[0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
时段 2758-2772 检测到的异常传感器：[0, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
时段 760-765：
→ 正确检测数：13/15
→ 漏检传感器：[5, 19]
→ 误检传感器：[0, 3, 6]
时段 1064-1298：
→ 正确检测数：12/13
→ 漏检传感器：[19]
→ 误检传感器：[0, 2, 3, 4, 5, 8]
时段 2758-2772：
→ 正确检测数：14/15
→ 漏检传感器：[19]
→ 误检传感器：[0, 3, 4]


In [ ]:
import numpy as np
from scipy.stats import pearsonr
from dtaidistance import dtw
import networkx as nx
from gensim.models import Word2Vec
from sklearn.cluster import KMeans
import pandas as pd

def strict_image_method(data, labels, window_size=5):
    """
    完全按照图片描述的三个步骤实现：
    1. 构建图结构（节点为时间窗口,边由两个条件：Pearson相关系数和DTW距离确定）
    2. 基于随机游走和Word2Vec实现DeepWalk嵌入
    3. 利用KMeans聚类并通过聚类较小的那一类判断异常窗口,再转换回原始时间点,
       并统计每个传感器的异常频率（使用Z-score计算）。
    """
    # 1. 构建图结构：节点为连续时间窗口
    G = nx.Graph()
    n_windows = len(data) - window_size + 1
    for i in range(n_windows):
        window = data[i:i+window_size]
        G.add_node(i, values=window)
    
    # 添加边（严格按两个条件判断）
    for i in range(n_windows):
        for j in range(i+1, min(i+20, n_windows)):  # 限制连接范围
            # 条件1：Pearson相关系数
            corr = pearsonr(G.nodes[i]['values'].flatten(), 
                            G.nodes[j]['values'].flatten())[0]
            # 条件2：DTW距离（这里先计算各窗口沿时间方向的均值序列）
            dtw_dist = dtw.distance(G.nodes[i]['values'].mean(axis=1),
                                    G.nodes[j]['values'].mean(axis=1))
            
            if abs(corr) > 0.6 and dtw_dist < 1.5:  # 严格阈值条件
                G.add_edge(i, j, weight=abs(corr))
    
    # 2. DeepWalk：利用随机游走获得节点序列,再用Word2Vec训练嵌入
    def random_walk(G, walk_length=15):
        walks = []
        for _ in range(10):  # 每个节点执行10次随机游走
            for node in G.nodes():
                walk = [str(node)]
                while len(walk) < walk_length:
                    current = int(walk[-1])
                    neighbors = list(G.neighbors(current))
                    if neighbors:
                        walk.append(str(np.random.choice(neighbors)))
                    else:
                        break
                walks.append(walk)
        return walks

    walks = random_walk(G)
    model = Word2Vec(walks, vector_size=32, window=5, min_count=0, sg=1)
    embeddings = np.array([model.wv[str(i)] for i in G.nodes()])
    
    # 3. 异常检测：使用KMeans聚类,并认为聚类数量较少的一类对应异常窗口
    kmeans = KMeans(n_clusters=2, random_state=42)
    clusters = kmeans.fit_predict(embeddings)
    # 选择窗口数目较少的聚类作为异常窗口
    anomalies = np.where(clusters == np.argmin(np.bincount(clusters)))[0]
    
    # 将异常窗口映射回原始时间点
    anomaly_windows = [i for i in anomalies if i in G.nodes()]
    anomaly_points = []
    for win in anomaly_windows:
        anomaly_points.extend(range(win, win+window_size))
    
    # 统计每个传感器的异常频率（仅统计标签为1的真实异常点）
    sensor_scores = np.zeros(data.shape[1])
    for point in set(anomaly_points):
        if point < len(labels) and labels[point] == 1:
            for sensor in range(data.shape[1]):
                # 计算局部Z-score
                mean_val = np.mean(data[max(0, point-100):point+1, sensor])
                std_val = np.std(data[max(0, point-100):point+1, sensor])
                sensor_scores[sensor] += abs(data[point, sensor] - mean_val) / (std_val + 1e-8)
    
    # 这里简单返回异常得分大于0的传感器（您可根据需求设定更复杂的阈值）
    return np.where(sensor_scores > 0)[0]

# 循环遍历 /data/processed/ 下的所有测试数据文件以omi-开头的文件
def get_test_files():
    import os
    files = os.listdir('data/processed/')
    test_files = [f for f in files if f.startswith('omi-') and f.endswith('_test.pkl')]
    return test_files



if __name__ == "__main__":





    # # 加载测试数据与标签
    # test_data = pd.read_pickle('data/processed/omi-1_test.pkl')
    # test_labels = pd.read_pickle('data/processed/omi-1_test_label.pkl')

    # # 如果读取结果为numpy数组,则转换为DataFrame
    # if isinstance(test_data, np.ndarray):
    #     test_data = pd.DataFrame(test_data)
    # if isinstance(test_labels, np.ndarray):
    #     test_labels = pd.DataFrame(test_labels, columns=["label"])
    
    # # 设定需要检测的时段
    # segments = [(760, 765), (1064, 1298), (2758, 2772)]
    # results = {}
    # for segment in segments:
    #     # 提取指定时段数据和对应标签（转换为numpy数组）
    #     seg_data = test_data.iloc[segment[0]:segment[1]+1].to_numpy()
    #     seg_labels = test_labels.iloc[segment[0]:segment[1]+1]["label"].to_numpy()
    #     anomalies = strict_image_method(seg_data, seg_labels)
    #     results[f"{segment[0]}-{segment[1]}"] = anomalies
    #     print(f"时段 {segment[0]}-{segment[1]} 检测到的异常传感器：{sorted(anomalies)}")
    
    # # 结果对比（示例）：与预期结果对比
    # expected = {
    #     "760-765": [4,5,7,8,9,10,11,12,13,14,15,16,17,18,19],
    #     "1064-1298": [6,7,9,10,11,12,13,14,15,16,17,18,19],
    #     "2758-2772": [5,6,7,8,9,10,11,12,13,14,15,16,17,18,19]
    # }
    
    # for seg in results:
    #     detected = set(results[seg])
    #     actual = set(expected[seg])
    #     print(f"时段 {seg}：")
    #     print(f"→ 正确检测数：{len(detected & actual)}/{len(actual)}")
    #     print(f"→ 漏检传感器：{sorted(actual - detected)}")
    #     print(f"→ 误检传感器：{sorted(detected - actual)}")
    #     print("="*50)


In [39]:
import os
import numpy as np
from scipy.stats import pearsonr
from dtaidistance import dtw
import networkx as nx
from gensim.models import Word2Vec
from sklearn.cluster import KMeans
import pandas as pd

def strict_image_method(data, labels, window_size=5):
    """
    根据描述实现异常检测：
    1. 构建图结构：节点为连续时间窗口（窗口内数据存储在节点属性 'values'），
       边的条件：窗口数据展平后的 Pearson 相关系数绝对值 > 0.6，
       且各窗口沿时间方向均值序列的 DTW 距离 < 1.5。
    2. 利用随机游走和 Word2Vec（DeepWalk 思想）获得每个窗口的低维嵌入表示。
    3. 用 KMeans 聚类（聚类数量较少的那一类视为异常窗口），
       将异常窗口映射回原始时间点，再对真实异常点（labels==1）计算局部 Z-score，
       返回异常传感器索引。
    """
    # 1. 构建图结构
    G = nx.Graph()
    n_windows = len(data) - window_size + 1
    if n_windows < 2:
        # 如果窗口数不足以构建至少2个节点，则直接返回空数组
        return np.array([])
    for i in range(n_windows):
        window = data[i:i+window_size]
        G.add_node(i, values=window)
    
    # 添加边：仅比较窗口 i 与 i+1 到 i+20 内的窗口
    for i in range(n_windows):
        for j in range(i+1, min(i+20, n_windows)):
            # 条件1：计算 Pearson 相关系数（将窗口展平后计算）
            corr = pearsonr(G.nodes[i]['values'].flatten(), 
                            G.nodes[j]['values'].flatten())[0]
            # 条件2：计算 DTW 距离（对窗口数据取沿时间方向的均值序列）
            dtw_dist = dtw.distance(G.nodes[i]['values'].mean(axis=1),
                                    G.nodes[j]['values'].mean(axis=1))
            if abs(corr) > 0.6 and dtw_dist < 1.5:
                G.add_edge(i, j, weight=abs(corr))
    
    # 2. DeepWalk：利用随机游走获得节点序列，再用 Word2Vec 训练得到嵌入
    def random_walk(G, walk_length=15):
        walks = []
        for _ in range(10):  # 每个节点执行10次随机游走
            for node in G.nodes():
                walk = [str(node)]
                while len(walk) < walk_length:
                    current = int(walk[-1])
                    neighbors = list(G.neighbors(current))
                    if neighbors:
                        walk.append(str(np.random.choice(neighbors)))
                    else:
                        break
                walks.append(walk)
        return walks

    walks = random_walk(G)
    model = Word2Vec(walks, vector_size=32, window=5, min_count=0, sg=1)
    embeddings = np.array([model.wv[str(i)] for i in G.nodes()])
    
    # 3. 异常检测：利用 KMeans 聚类，选择窗口数较少的那一类为异常窗口
    if embeddings.shape[0] < 2:
        return np.array([])
    kmeans = KMeans(n_clusters=2, random_state=42)
    clusters = kmeans.fit_predict(embeddings)
    anomalies = np.where(clusters == np.argmin(np.bincount(clusters)))[0]
    
    # 将异常窗口映射回原始时间点
    anomaly_windows = [i for i in anomalies if i in G.nodes()]
    anomaly_points = []
    for win in anomaly_windows:
        anomaly_points.extend(range(win, win+window_size))
    
    # 针对每个异常点（仅统计真实异常点，即 labels==1），计算各传感器局部 Z-score 异常得分
    sensor_scores = np.zeros(data.shape[1])
    for point in set(anomaly_points):
        if point < len(labels) and labels[point] == 1:
            for sensor in range(data.shape[1]):
                local_data = data[max(0, point-100):point+1, sensor]
                mean_val = np.mean(local_data)
                std_val = np.std(local_data)
                sensor_scores[sensor] += abs(data[point, sensor] - mean_val) / (std_val + 1e-8)
    return np.where(sensor_scores > 0)[0]

def process_omi_file(txt_file_path, test_data, test_labels):
    """
    处理单个 omi- 开头的 txt 文件：
      1. 解析文件中每行的时段和期望异常传感器；
      2. 对每个时段从 test_data 和 test_labels 中提取数据，调用 strict_image_method 进行检测；
      3. 输出检测结果与预期结果的对比信息。
    """
    print("="*60)
    print("Processing file:", txt_file_path)
    with open(txt_file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    
    for line in lines:
        line = line.strip()
        if not line:
            continue
        if ':' not in line:
            print("格式错误，缺少冒号：", line)
            continue
        time_part, sensor_part = line.split(":", 1)
        if '-' not in time_part:
            print("格式错误，缺少 '-'：", line)
            continue
        try:
            start_str, end_str = time_part.split('-')
            start_idx = int(start_str)
            end_idx = int(end_str)
        except ValueError:
            print("时段解析错误：", line)
            continue
        if sensor_part.strip():
            try:
                expected_sensors = [int(x) for x in sensor_part.split(',') if x.strip() != '']
            except ValueError:
                print("传感器列表解析错误：", line)
                expected_sensors = []
        else:
            expected_sensors = []
        
        # 提取该时段数据（转换为 numpy 数组）
        seg_data = test_data.iloc[start_idx:end_idx+1].to_numpy()
        seg_labels = test_labels.iloc[start_idx:end_idx+1]["label"].to_numpy()
        
        detected_sensors = strict_image_method(seg_data, seg_labels)
        
        detected_set = set(detected_sensors)
        expected_set = set(expected_sensors)
        true_positives = detected_set & expected_set
        false_negatives = expected_set - detected_set
        false_positives = detected_set - expected_set
        
        print(f"时段 {start_idx}-{end_idx}:")
        print(f"  期望异常传感器: {sorted(expected_sensors)}")
        print(f"  检测到异常传感器: {sorted(detected_sensors)}")
        print(f"  命中: {sorted(true_positives)}")
        print(f"  漏检: {sorted(false_negatives)}")
        print(f"  误检: {sorted(false_positives)}")
        print("-"*50)

if __name__ == "__main__":
    # 指定包含 omi- 开头 txt 文件的文件夹（只处理 omi- 文件）
    label_folder = r"data\interpretation_label"
    txt_files = [f for f in os.listdir(label_folder)
                 if f.startswith("omi-") and f.endswith(".txt")]
    
    # 遍历每个 omi- 文件
    for txt_filename in sorted(txt_files):
        # 根据文件名前缀（例如 "omi-1"）构造对应的测试数据和标签路径
        prefix = txt_filename.split(".")[0]  # 如 "omi-1"
        test_data_path = os.path.join("data/processed", f"{prefix}_test.pkl")
        test_labels_path = os.path.join("data/processed", f"{prefix}_test_label.pkl")
        
        # 检查对应的测试数据和标签是否存在
        if not (os.path.exists(test_data_path) and os.path.exists(test_labels_path)):
            print(f"缺少对应的测试数据或标签文件：{prefix}")
            continue
        
        # 加载测试数据和标签
        test_data = pd.read_pickle(test_data_path)
        test_labels = pd.read_pickle(test_labels_path)
        if isinstance(test_data, np.ndarray):
            test_data = pd.DataFrame(test_data)
        if isinstance(test_labels, np.ndarray):
            test_labels = pd.DataFrame(test_labels, columns=["label"])
        elif isinstance(test_labels, pd.Series):
            test_labels = test_labels.to_frame(name="label")
        
        file_path = os.path.join(label_folder, txt_filename)
        process_omi_file(file_path, test_data, test_labels)


Processing file: data\interpretation_label\omi-1.txt
时段 760-765:
  期望异常传感器: [4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
  检测到异常传感器: [0, 3, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
  命中: [4, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
  漏检: [5, 19]
  误检: [0, 3, 6]
--------------------------------------------------
时段 1064-1298:
  期望异常传感器: [6, 7, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
  检测到异常传感器: [0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
  命中: [6, 7, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
  漏检: [19]
  误检: [0, 2, 3, 4, 5, 8]
--------------------------------------------------
时段 2758-2772:
  期望异常传感器: [5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
  检测到异常传感器: [0, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
  命中: [5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
  漏检: [19]
  误检: [0, 3, 4]
--------------------------------------------------
时段 2874-2885:
  期望异常传感器: [7, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
  检测到异常传感器: [0

In [41]:
import os
import numpy as np
from scipy.stats import pearsonr
from dtaidistance import dtw
import networkx as nx
from gensim.models import Word2Vec
from sklearn.cluster import KMeans
import pandas as pd

def strict_image_method(data, labels, window_size=5):
    """
    根据描述实现异常检测：
    1. 构建图结构：节点为连续时间窗口（窗口内数据存储在节点属性 'values'），
       边的条件：窗口数据展平后的 Pearson 相关系数绝对值 > 0.6，
       且各窗口沿时间方向均值序列的 DTW 距离 < 1.5。
    2. 利用随机游走和 Word2Vec（DeepWalk 思想）获得每个窗口的低维嵌入表示。
    3. 用 KMeans 聚类（聚类数量较少的那一类视为异常窗口），
       将异常窗口映射回原始时间点，再对真实异常点（labels==1）计算局部 Z-score，
       返回异常传感器索引。
    """
    # 1. 构建图结构
    G = nx.Graph()
    n_windows = len(data) - window_size + 1
    if n_windows < 2:
        # 窗口数不足2则直接返回空数组
        return np.array([])
    for i in range(n_windows):
        window = data[i:i+window_size]
        G.add_node(i, values=window)
    
    # 添加边：仅比较窗口 i 与 i+1 到 i+20 内的窗口
    for i in range(n_windows):
        for j in range(i+1, min(i+20, n_windows)):
            # 条件1：计算 Pearson 相关系数（展平后计算）
            corr = pearsonr(G.nodes[i]['values'].flatten(), 
                            G.nodes[j]['values'].flatten())[0]
            # 条件2：计算 DTW 距离（对窗口数据取沿时间方向的均值序列）
            dtw_dist = dtw.distance(G.nodes[i]['values'].mean(axis=1),
                                    G.nodes[j]['values'].mean(axis=1))
            if abs(corr) > 0.6 and dtw_dist < 1.5:
                G.add_edge(i, j, weight=abs(corr))
    
    # 2. DeepWalk：利用随机游走获得节点序列，再用 Word2Vec 训练嵌入
    def random_walk(G, walk_length=15):
        walks = []
        for _ in range(10):  # 每个节点执行10次随机游走
            for node in G.nodes():
                walk = [str(node)]
                while len(walk) < walk_length:
                    current = int(walk[-1])
                    neighbors = list(G.neighbors(current))
                    if neighbors:
                        walk.append(str(np.random.choice(neighbors)))
                    else:
                        break
                walks.append(walk)
        return walks

    walks = random_walk(G)
    model = Word2Vec(walks, vector_size=32, window=5, min_count=0, sg=1)
    embeddings = np.array([model.wv[str(i)] for i in G.nodes()])
    
    # 3. 异常检测：利用 KMeans 聚类，选择数量较少的簇作为异常窗口
    if embeddings.shape[0] < 2:
        return np.array([])
    kmeans = KMeans(n_clusters=2, random_state=42)
    clusters = kmeans.fit_predict(embeddings)
    anomalies = np.where(clusters == np.argmin(np.bincount(clusters)))[0]
    
    # 将异常窗口映射回原始时间点
    anomaly_windows = [i for i in anomalies if i in G.nodes()]
    anomaly_points = []
    for win in anomaly_windows:
        anomaly_points.extend(range(win, win+window_size))
    
    # 针对每个异常点（仅统计真实异常点 labels==1），计算各传感器局部 Z-score 异常得分
    sensor_scores = np.zeros(data.shape[1])
    for point in set(anomaly_points):
        if point < len(labels) and labels[point] == 1:
            for sensor in range(data.shape[1]):
                local_data = data[max(0, point-100):point+1, sensor]
                mean_val = np.mean(local_data)
                std_val = np.std(local_data)
                sensor_scores[sensor] += abs(data[point, sensor] - mean_val) / (std_val + 1e-8)
    return np.where(sensor_scores > 0)[0]

def process_omi_file(txt_file_path, test_data, test_labels):
    """
    处理单个 omi- 开头的 txt 文件：
      1. 解析文件中每行的时段和期望异常传感器；
      2. 对每个时段从 test_data 和 test_labels 中提取数据，调用 strict_image_method 进行检测；
      3. 输出检测结果与预期结果的对比信息，同时计算正确检测数、漏检比例、误检比例。
    """
    print("="*60)
    print("Processing file:", txt_file_path)
    with open(txt_file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    
    for line in lines:
        line = line.strip()
        if not line:
            continue
        if ':' not in line:
            print("格式错误，缺少冒号：", line)
            continue
        time_part, sensor_part = line.split(":", 1)
        if '-' not in time_part:
            print("格式错误，缺少 '-'：", line)
            continue
        try:
            start_str, end_str = time_part.split('-')
            start_idx = int(start_str)
            end_idx = int(end_str)
        except ValueError:
            print("时段解析错误：", line)
            continue
        if sensor_part.strip():
            try:
                expected_sensors = [int(x) for x in sensor_part.split(',') if x.strip() != '']
            except ValueError:
                print("传感器列表解析错误：", line)
                expected_sensors = []
        else:
            expected_sensors = []
        
        # 提取该时段数据（转换为 numpy 数组）
        seg_data = test_data.iloc[start_idx:end_idx+1].to_numpy()
        seg_labels = test_labels.iloc[start_idx:end_idx+1]["label"].to_numpy()
        
        detected_sensors = strict_image_method(seg_data, seg_labels)
        
        detected_set = set(detected_sensors)
        expected_set = set(expected_sensors)
        true_positives = detected_set & expected_set
        false_negatives = expected_set - detected_set
        false_positives = detected_set - expected_set
        
        # 计算指标
        correct_count = len(true_positives)
        miss_ratio = len(false_negatives) / len(expected_set) if len(expected_set) > 0 else 0
        false_ratio = len(false_positives) / len(detected_set) if len(detected_set) > 0 else 0
        
        print(f"时段 {start_idx}-{end_idx}:")
        print(f"  期望异常传感器: {sorted(expected_sensors)}")
        print(f"  检测到异常传感器: {sorted(detected_sensors)}")
        print(f"  命中: {sorted(true_positives)}")
        print(f"  漏检: {sorted(false_negatives)}")
        print(f"  误检: {sorted(false_positives)}")
        print(f"  正确检测数: {correct_count}")
        print(f"  漏检比例: {miss_ratio:.2f}")
        print(f"  误检比例: {false_ratio:.2f}")
        print("-"*50)

if __name__ == "__main__":
    # 指定包含 omi- 开头 txt 文件的文件夹（只处理 omi- 文件）
    label_folder = r"data\interpretation_label"
    txt_files = [f for f in os.listdir(label_folder)
                 if f.startswith("omi-") and f.endswith(".txt")]
    
    # 遍历每个 omi- 文件，根据文件名前缀加载对应的测试数据和测试标签文件
    for txt_filename in sorted(txt_files):
        # 根据文件名前缀（如 "omi-1"）构造测试数据和标签文件路径
        prefix = txt_filename.split(".")[0]  # 例如 "omi-1"
        test_data_path = os.path.join("data/processed", f"{prefix}_test.pkl")
        test_labels_path = os.path.join("data/processed", f"{prefix}_test_label.pkl")
        
        # 检查对应文件是否存在
        if not (os.path.exists(test_data_path) and os.path.exists(test_labels_path)):
            print(f"缺少对应的测试数据或标签文件：{prefix}")
            continue
        
        test_data = pd.read_pickle(test_data_path)
        test_labels = pd.read_pickle(test_labels_path)
        if isinstance(test_data, np.ndarray):
            test_data = pd.DataFrame(test_data)
        if isinstance(test_labels, np.ndarray):
            test_labels = pd.DataFrame(test_labels, columns=["label"])
        elif isinstance(test_labels, pd.Series):
            test_labels = test_labels.to_frame(name="label")
        
        file_path = os.path.join(label_folder, txt_filename)
        process_omi_file(file_path, test_data, test_labels)


Processing file: data\interpretation_label\omi-1.txt
时段 760-765:
  期望异常传感器: [4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
  检测到异常传感器: [0, 3, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
  命中: [4, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
  漏检: [5, 19]
  误检: [0, 3, 6]
  正确检测数: 13
  漏检比例: 0.13
  误检比例: 0.19
--------------------------------------------------
时段 1064-1298:
  期望异常传感器: [6, 7, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
  检测到异常传感器: [0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
  命中: [6, 7, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
  漏检: [19]
  误检: [0, 2, 3, 4, 5, 8]
  正确检测数: 12
  漏检比例: 0.08
  误检比例: 0.33
--------------------------------------------------
时段 2758-2772:
  期望异常传感器: [5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
  检测到异常传感器: [0, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
  命中: [5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
  漏检: [19]
  误检: [0, 3, 4]
  正确检测数: 14
  漏检比例: 0.07
  误检比例: 0.18
-------------------

In [42]:
import os
import numpy as np
from scipy.stats import pearsonr
from dtaidistance import dtw
import networkx as nx
from gensim.models import Word2Vec
from sklearn.cluster import KMeans
import pandas as pd

def enhanced_strict_image_method(data, labels, window_size=5):
    """严格遵循图片方法的增强实现"""
    # 1. 构建图结构（节点=时间窗口）
    G = nx.Graph()
    n_windows = len(data) - window_size + 1
    
    # 添加节点（存储窗口数据和统计特征）
    for i in range(n_windows):
        window = data[i:i+window_size]
        features = [
            np.mean(window, axis=0),
            np.std(window, axis=0),
            np.max(window, axis=0) - np.min(window, axis=0)
        ]
        G.add_node(i, values=window, features=np.concatenate(features))
    
    # 添加边（严格双条件）
    for i in range(n_windows):
        for j in range(i+1, min(i+20, n_windows)):
            # 条件1：Pearson相关系数（绝对值）
            corr = abs(pearsonr(G.nodes[i]['values'].flatten(), 
                              G.nodes[j]['values'].flatten())[0])
            # 条件2：DTW距离（各传感器均值序列）
            dtw_dist = dtw.distance(G.nodes[i]['values'].mean(axis=1),
                                   G.nodes[j]['values'].mean(axis=1))
            
            if corr > 0.7 and dtw_dist < 1.2:  # 调整后的阈值
                G.add_edge(i, j, weight=corr*(1-dtw_dist/1.2))
    
    # 2. 改进的DeepWalk（带重启机制）
    def enhanced_random_walk(G, walk_length=15, restart_prob=0.1):
        walks = []
        for _ in range(15):  # 每个节点游走15次
            for node in G.nodes():
                walk = [str(node)]
                while len(walk) < walk_length:
                    if np.random.rand() < restart_prob:
                        walk.append(str(node))  # 重启机制
                    else:
                        current = int(walk[-1])
                        neighbors = list(G.neighbors(current))
                        if neighbors:
                            weights = [G[current][n]['weight'] for n in neighbors]
                            next_node = np.random.choice(neighbors, p=np.array(weights)/sum(weights))
                            walk.append(str(next_node))
                        else:
                            break
                walks.append(walk)
        return walks
    
    walks = enhanced_random_walk(G)
    model = Word2Vec(walks, vector_size=64, window=5, min_count=0, sg=1, workers=4)
    embeddings = np.array([model.wv[str(i)] for i in G.nodes()])
    
    # 3. 异常检测（双重验证）
    # 聚类检测
    kmeans = KMeans(n_clusters=2)
    clusters = kmeans.fit_predict(embeddings)
    cluster_anomalies = np.where(clusters == np.argmin(np.bincount(clusters)))[0]
    
    # 重构误差检测
    recon_scores = np.zeros(n_windows)
    for i in G.nodes():
        neighbors = list(G.neighbors(i))
        if neighbors:
            pred = np.mean([model.wv[str(n)] * G[i][n]['weight'] for n in neighbors], axis=0)
            recon_scores[i] = np.linalg.norm(model.wv[str(i)] - pred)
    
    # 综合判定
    anomaly_windows = [
        i for i in cluster_anomalies 
        if recon_scores[i] > np.percentile(recon_scores, 75)
    ]
    
    # 4. 传感器级异常定位
    sensor_scores = np.zeros(data.shape[1])
    for win in anomaly_windows:
        for t in range(win, win+window_size):
            if t < len(labels) and labels[t] == 1:
                for sensor in range(data.shape[1]):
                    local_mean = np.mean(data[max(0,t-50):t+1, sensor])
                    local_std = np.std(data[max(0,t-50):t+1, sensor]) + 1e-8
                    sensor_scores[sensor] += abs(data[t, sensor] - local_mean) / local_std
    
    # 动态阈值选择
    threshold = np.percentile(sensor_scores[sensor_scores > 0], 80) if np.any(sensor_scores > 0) else 0
    return np.where(sensor_scores > threshold)[0]

def process_omi_file(txt_file_path, test_data, test_labels):
    """改进的结果处理函数"""
    print("="*60)
    print("Processing:", os.path.basename(txt_file_path))
    with open(txt_file_path, 'r') as f:
        lines = [line.strip() for line in f if line.strip()]
    
    total_metrics = {'correct':0, 'expected':0, 'detected':0}
    
    for line in lines:
        try:
            # 解析时段和预期传感器
            time_part, sensor_part = line.split(':', 1)
            start, end = map(int, time_part.split('-'))
            expected = [int(s) for s in sensor_part.split(',') if s.strip()]
            
            # 执行检测
            seg_data = test_data.iloc[start:end+1].values
            seg_labels = test_labels.iloc[start:end+1].values.flatten()
            detected = enhanced_strict_image_method(seg_data, seg_labels)
            
            # 计算指标
            expected_set = set(expected)
            detected_set = set(detected)
            correct = expected_set & detected_set
            missing = expected_set - detected_set
            false_alarms = detected_set - expected_set
            
            # 更新全局统计
            total_metrics['correct'] += len(correct)
            total_metrics['expected'] += len(expected_set)
            total_metrics['detected'] += len(detected_set)
            
            # 打印结果
            print(f"\n时段 {start}-{end}:")
            print(f"预期异常: {sorted(expected_set)}")
            print(f"检测结果: {sorted(detected_set)}")
            print(f"正确检测: {sorted(correct)}")
            print(f"漏检: {sorted(missing)}")
            print(f"误检: {sorted(false_alarms)}")
            print(f"准确率: {len(correct)/len(expected_set):.1%}") if expected_set else print("无预期异常")
            
        except Exception as e:
            print(f"处理行时出错: {line}\n错误: {str(e)}")
    
    # 打印总体统计
    if total_metrics['expected'] > 0:
        print("\n" + "="*30)
        print(f"总体统计:")
        print(f"总正确检测: {total_metrics['correct']}/{total_metrics['expected']}")
        print(f"召回率: {total_metrics['correct']/total_metrics['expected']:.1%}")
        print(f"精确率: {total_metrics['correct']/total_metrics['detected']:.1%}") if total_metrics['detected'] else print("无检测结果")

if __name__ == "__main__":
    # 配置数据路径
    data_dir = "data/processed"
    label_dir = "data/interpretation_label"
    
    # 处理所有omi文件
    for file in os.listdir(label_dir):
        if file.startswith('omi-') and file.endswith('.txt'):
            prefix = file.split('.')[0]
            try:
                # 加载数据
                data = pd.read_pickle(f"{data_dir}/{prefix}_test.pkl")
                labels = pd.read_pickle(f"{data_dir}/{prefix}_test_label.pkl")
                
                # 确保数据格式正确
                if isinstance(data, pd.DataFrame):
                    data = data.values
                if isinstance(labels, pd.Series):
                    labels = labels.values
                elif isinstance(labels, pd.DataFrame):
                    labels = labels.iloc[:,0].values
                
                # 执行处理
                print("\n" + "="*60)
                print(f"处理文件: {file}")
                process_omi_file(f"{label_dir}/{file}", pd.DataFrame(data), pd.DataFrame(labels))
                
            except Exception as e:
                print(f"处理文件{file}时出错: {str(e)}")


处理文件: omi-1.txt
Processing: omi-1.txt

时段 760-765:
预期异常: [4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
检测结果: []
正确检测: []
漏检: [4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
误检: []
准确率: 0.0%

时段 1064-1298:
预期异常: [6, 7, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
检测结果: [5, 13, 16, 17]
正确检测: [13, 16, 17]
漏检: [6, 7, 9, 10, 11, 12, 14, 15, 18, 19]
误检: [5]
准确率: 23.1%

时段 2758-2772:
预期异常: [5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
检测结果: []
正确检测: []
漏检: [5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
误检: []
准确率: 0.0%

时段 2874-2885:
预期异常: [7, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
检测结果: [4, 5, 6, 18]
正确检测: [18]
漏检: [7, 9, 10, 11, 12, 13, 14, 15, 16, 17]
误检: [4, 5, 6]
准确率: 9.1%

时段 3012-3025:
预期异常: [7, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
检测结果: [7, 18]
正确检测: [7, 18]
漏检: [10, 11, 12, 13, 14, 15, 16, 17, 19]
误检: []
准确率: 18.2%

时段 3160-3305:
预期异常: [8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
检测结果: [4, 12, 14, 15]
正确检测: [12, 14, 15]
漏检: [8, 9, 10, 11, 13, 

In [43]:
import os
import numpy as np
from scipy.stats import pearsonr
from dtaidistance import dtw
import networkx as nx
from gensim.models import Word2Vec
from sklearn.cluster import KMeans
import pandas as pd

def robust_zscore(x, median_val, mad):
    return np.abs(x - median_val) / (mad + 1e-8)

def strict_image_method(data, labels, window_size=5):
    """
    改进后的异常检测方法：
    1. 构建图结构：节点为连续时间窗口（窗口内数据存储在节点属性 'values'），
       边的条件：窗口展平后的 Pearson 相关系数绝对值 > 0.65，
       且各窗口沿时间方向均值序列的 DTW 距离 < 1.6。
    2. 利用随机游走和 Word2Vec（DeepWalk 思想）获得每个窗口的低维嵌入表示，
       随机游走次数增加到20次，每个游走步长延长到20。
    3. 用 KMeans 聚类（聚类数设为2，数量较少的簇视为异常窗口），
       将异常窗口映射回原始时间点，再对真实异常点（labels==1）利用鲁棒统计（中位数和 MAD）计算局部异常得分，
       返回异常传感器索引（得分大于0）。
    """
    # 1. 构建图结构
    G = nx.Graph()
    n_windows = len(data) - window_size + 1
    if n_windows < 2:
        return np.array([])
    for i in range(n_windows):
        window = data[i:i+window_size]
        G.add_node(i, values=window)
    
    # 添加边：仅比较窗口 i 与 i+1 到 i+20 内的窗口
    for i in range(n_windows):
        for j in range(i+1, min(i+20, n_windows)):
            # 条件1：计算 Pearson 相关系数（展平后计算）
            corr = pearsonr(G.nodes[i]['values'].flatten(), 
                            G.nodes[j]['values'].flatten())[0]
            # 条件2：计算 DTW 距离（使用窗口数据沿时间方向的均值序列）
            dtw_dist = dtw.distance(G.nodes[i]['values'].mean(axis=1),
                                    G.nodes[j]['values'].mean(axis=1))
            if abs(corr) > 0.65 and dtw_dist < 1.6:
                G.add_edge(i, j, weight=abs(corr))
    
    # 2. DeepWalk：利用随机游走获得节点序列，再用 Word2Vec 得到嵌入
    def random_walk(G, walk_length=20, num_walks=20):
        walks = []
        for _ in range(num_walks):  # 每个节点执行20次随机游走
            for node in G.nodes():
                walk = [str(node)]
                while len(walk) < walk_length:
                    current = int(walk[-1])
                    neighbors = list(G.neighbors(current))
                    if neighbors:
                        walk.append(str(np.random.choice(neighbors)))
                    else:
                        break
                walks.append(walk)
        return walks

    walks = random_walk(G)
    model = Word2Vec(walks, vector_size=32, window=5, min_count=0, sg=1)
    embeddings = np.array([model.wv[str(i)] for i in G.nodes()])
    
    # 3. 异常检测：利用 KMeans 聚类，选择窗口数较少的簇作为异常窗口
    if embeddings.shape[0] < 2:
        return np.array([])
    kmeans = KMeans(n_clusters=2, random_state=42)
    clusters = kmeans.fit_predict(embeddings)
    anomalies = np.where(clusters == np.argmin(np.bincount(clusters)))[0]
    
    # 将异常窗口映射回原始时间点
    anomaly_windows = [i for i in anomalies if i in G.nodes()]
    anomaly_points = []
    for win in anomaly_windows:
        anomaly_points.extend(range(win, win+window_size))
    
    # 对于每个异常点（仅统计真实异常点，即 labels==1），使用鲁棒统计计算局部异常得分
    sensor_scores = np.zeros(data.shape[1])
    for point in set(anomaly_points):
        if point < len(labels) and labels[point] == 1:
            for sensor in range(data.shape[1]):
                local_data = data[max(0, point-100):point+1, sensor]
                median_val = np.median(local_data)
                mad = np.median(np.abs(local_data - median_val))
                sensor_scores[sensor] += robust_zscore(data[point, sensor], median_val, mad)
    return np.where(sensor_scores > 0)[0]

def process_omi_file(txt_file_path, test_data, test_labels):
    """
    处理单个 omi- 开头的 txt 文件：
    1. 解析文件中每行的时段和期望异常传感器；
    2. 对每个时段从 test_data 和 test_labels 中提取数据，调用 strict_image_method 进行检测；
    3. 输出检测结果与期望结果对比信息，同时计算正确检测数、漏检比例和误检比例。
    """
    print("="*60)
    print("Processing file:", txt_file_path)
    with open(txt_file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    
    for line in lines:
        line = line.strip()
        if not line:
            continue
        if ':' not in line:
            print("格式错误，缺少冒号：", line)
            continue
        time_part, sensor_part = line.split(":", 1)
        if '-' not in time_part:
            print("格式错误，缺少 '-'：", line)
            continue
        try:
            start_str, end_str = time_part.split('-')
            start_idx = int(start_str)
            end_idx = int(end_str)
        except ValueError:
            print("时段解析错误：", line)
            continue
        if sensor_part.strip():
            try:
                expected_sensors = [int(x) for x in sensor_part.split(',') if x.strip() != '']
            except ValueError:
                print("传感器列表解析错误：", line)
                expected_sensors = []
        else:
            expected_sensors = []
        
        # 提取该时段数据（转换为 numpy 数组）
        seg_data = test_data.iloc[start_idx:end_idx+1].to_numpy()
        seg_labels = test_labels.iloc[start_idx:end_idx+1]["label"].to_numpy()
        
        detected_sensors = strict_image_method(seg_data, seg_labels)
        
        detected_set = set(detected_sensors)
        expected_set = set(expected_sensors)
        true_positives = detected_set & expected_set
        false_negatives = expected_set - detected_set
        false_positives = detected_set - expected_set
        
        correct_count = len(true_positives)
        miss_ratio = len(false_negatives) / len(expected_set) if len(expected_set) > 0 else 0
        false_ratio = len(false_positives) / len(detected_set) if len(detected_set) > 0 else 0
        
        print(f"时段 {start_idx}-{end_idx}:")
        print(f"  期望异常传感器: {sorted(expected_sensors)}")
        print(f"  检测到异常传感器: {sorted(detected_sensors)}")
        print(f"  命中: {sorted(true_positives)}")
        print(f"  漏检: {sorted(false_negatives)}")
        print(f"  误检: {sorted(false_positives)}")
        print(f"  正确检测数: {correct_count}")
        print(f"  漏检比例: {miss_ratio:.2f}")
        print(f"  误检比例: {false_ratio:.2f}")
        print("-"*50)

if __name__ == "__main__":
    # 指定包含 omi- 开头 txt 文件的文件夹（只处理 omi- 文件）
    label_folder = r"data\interpretation_label"
    txt_files = [f for f in os.listdir(label_folder)
                 if f.startswith("omi-") and f.endswith(".txt")]
    
    # 遍历每个 omi- 文件，根据文件名前缀加载对应的测试数据和测试标签文件
    for txt_filename in sorted(txt_files):
        prefix = txt_filename.split(".")[0]  # 例如 "omi-1"
        test_data_path = os.path.join("data/processed", f"{prefix}_test.pkl")
        test_labels_path = os.path.join("data/processed", f"{prefix}_test_label.pkl")
        
        if not (os.path.exists(test_data_path) and os.path.exists(test_labels_path)):
            print(f"缺少对应的测试数据或标签文件：{prefix}")
            continue
        
        test_data = pd.read_pickle(test_data_path)
        test_labels = pd.read_pickle(test_labels_path)
        if isinstance(test_data, np.ndarray):
            test_data = pd.DataFrame(test_data)
        if isinstance(test_labels, np.ndarray):
            test_labels = pd.DataFrame(test_labels, columns=["label"])
        elif isinstance(test_labels, pd.Series):
            test_labels = test_labels.to_frame(name="label")
        
        file_path = os.path.join(label_folder, txt_filename)
        process_omi_file(file_path, test_data, test_labels)


Processing file: data\interpretation_label\omi-1.txt
时段 760-765:
  期望异常传感器: [4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
  检测到异常传感器: [0, 3, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
  命中: [4, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
  漏检: [5, 19]
  误检: [0, 3, 6]
  正确检测数: 13
  漏检比例: 0.13
  误检比例: 0.19
--------------------------------------------------
时段 1064-1298:
  期望异常传感器: [6, 7, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
  检测到异常传感器: [0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
  命中: [6, 7, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
  漏检: [19]
  误检: [0, 2, 3, 4, 5, 8]
  正确检测数: 12
  漏检比例: 0.08
  误检比例: 0.33
--------------------------------------------------
时段 2758-2772:
  期望异常传感器: [5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
  检测到异常传感器: [0, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
  命中: [5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
  漏检: [19]
  误检: [0, 3, 4]
  正确检测数: 14
  漏检比例: 0.07
  误检比例: 0.18
-------------------

In [44]:
import os
import numpy as np
from scipy.stats import pearsonr
from dtaidistance import dtw
import networkx as nx
from gensim.models import Word2Vec
from sklearn.cluster import KMeans
import pandas as pd

def robust_zscore(x, median_val, mad):
    return np.abs(x - median_val) / (mad + 1e-8)

def strict_image_method(data, labels, window_size=5):
    """
    改进后的异常检测方法：
    1. 构建图结构：节点为连续时间窗口（窗口内数据存储在节点属性 'values'），
       边的条件：窗口展平后的 Pearson 相关系数绝对值 > 0.65，
       且各窗口沿时间方向均值序列的 DTW 距离 < 1.6。
    2. 利用随机游走和 Word2Vec（DeepWalk 思想）获得每个窗口的低维嵌入表示，
       随机游走次数增加到20次，每个游走步长延长到20。
    3. 用 KMeans 聚类（聚类数设为2，数量较少的簇视为异常窗口），
       将异常窗口映射回原始时间点，再对真实异常点（labels==1）利用鲁棒统计（中位数和 MAD）计算局部异常得分，
       返回异常传感器索引（得分大于0）。
    """
    # 1. 构建图结构
    G = nx.Graph()
    n_windows = len(data) - window_size + 1
    if n_windows < 2:
        return np.array([])
    for i in range(n_windows):
        window = data[i:i+window_size]
        G.add_node(i, values=window)
    
    # 添加边：仅比较窗口 i 与 i+1 到 i+20 内的窗口
    for i in range(n_windows):
        for j in range(i+1, min(i+20, n_windows)):
            corr = pearsonr(G.nodes[i]['values'].flatten(), 
                            G.nodes[j]['values'].flatten())[0]
            dtw_dist = dtw.distance(G.nodes[i]['values'].mean(axis=1),
                                    G.nodes[j]['values'].mean(axis=1))
            if abs(corr) > 0.65 and dtw_dist < 1.6:
                G.add_edge(i, j, weight=abs(corr))
    
    # 2. DeepWalk：随机游走获得节点序列，再用 Word2Vec 得到嵌入
    def random_walk(G, walk_length=20, num_walks=20):
        walks = []
        for _ in range(num_walks):
            for node in G.nodes():
                walk = [str(node)]
                while len(walk) < walk_length:
                    current = int(walk[-1])
                    neighbors = list(G.neighbors(current))
                    if neighbors:
                        walk.append(str(np.random.choice(neighbors)))
                    else:
                        break
                walks.append(walk)
        return walks

    walks = random_walk(G)
    model = Word2Vec(walks, vector_size=32, window=5, min_count=0, sg=1)
    embeddings = np.array([model.wv[str(i)] for i in G.nodes()])
    
    # 3. 异常检测：利用 KMeans 聚类，选择数量较少的簇作为异常窗口
    if embeddings.shape[0] < 2:
        return np.array([])
    kmeans = KMeans(n_clusters=2, random_state=42)
    clusters = kmeans.fit_predict(embeddings)
    anomalies = np.where(clusters == np.argmin(np.bincount(clusters)))[0]
    
    # 将异常窗口映射回原始时间点
    anomaly_windows = [i for i in anomalies if i in G.nodes()]
    anomaly_points = []
    for win in anomaly_windows:
        anomaly_points.extend(range(win, win+window_size))
    
    # 对于每个异常点（仅统计真实异常点 labels==1），使用鲁棒统计计算局部异常得分
    sensor_scores = np.zeros(data.shape[1])
    for point in set(anomaly_points):
        if point < len(labels) and labels[point] == 1:
            for sensor in range(data.shape[1]):
                local_data = data[max(0, point-100):point+1, sensor]
                median_val = np.median(local_data)
                mad = np.median(np.abs(local_data - median_val))
                sensor_scores[sensor] += robust_zscore(data[point, sensor], median_val, mad)
    return np.where(sensor_scores > 0)[0]

def process_omi_file(txt_file_path, test_data, test_labels):
    """
    处理单个 omi- 文件：
      1. 解析每行记录（格式：start-end:expected_sensor_list）；
      2. 对每个时段从 test_data 和 test_labels 中提取数据，调用 strict_image_method 进行检测；
      3. 输出每个时段的检测结果及对比，同时返回该文件所有时段的统计数据。
    返回一个字典，包含：
      - expected_sum: 累计期望异常传感器数
      - true_positive_sum: 累计命中数
      - detected_sum: 累计检测到的异常传感器数
    """
    print("="*60)
    print("Processing file:", txt_file_path)
    with open(txt_file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    
    expected_sum = 0
    true_positive_sum = 0
    detected_sum = 0
    
    for line in lines:
        line = line.strip()
        if not line:
            continue
        if ':' not in line:
            print("格式错误，缺少冒号：", line)
            continue
        time_part, sensor_part = line.split(":", 1)
        if '-' not in time_part:
            print("格式错误，缺少 '-'：", line)
            continue
        try:
            start_str, end_str = time_part.split('-')
            start_idx = int(start_str)
            end_idx = int(end_str)
        except ValueError:
            print("时段解析错误：", line)
            continue
        if sensor_part.strip():
            try:
                expected_sensors = [int(x) for x in sensor_part.split(',') if x.strip() != '']
            except ValueError:
                print("传感器列表解析错误：", line)
                expected_sensors = []
        else:
            expected_sensors = []
        
        # 提取该时段数据（转换为 numpy 数组）
        seg_data = test_data.iloc[start_idx:end_idx+1].to_numpy()
        seg_labels = test_labels.iloc[start_idx:end_idx+1]["label"].to_numpy()
        
        detected_sensors = strict_image_method(seg_data, seg_labels)
        
        detected_set = set(detected_sensors)
        expected_set = set(expected_sensors)
        true_positives = detected_set & expected_set
        false_negatives = expected_set - detected_set
        false_positives = detected_set - expected_set
        
        expected_count = len(expected_set)
        true_positive_count = len(true_positives)
        detected_count = len(detected_set)
        
        # 累加全局统计
        expected_sum += expected_count
        true_positive_sum += true_positive_count
        detected_sum += detected_count
        
        miss_ratio = len(false_negatives) / expected_count if expected_count > 0 else 0
        false_ratio = len(false_positives) / detected_count if detected_count > 0 else 0
        
        print(f"时段 {start_idx}-{end_idx}:")
        print(f"  期望异常传感器: {sorted(expected_set)}")
        print(f"  检测到异常传感器: {sorted(detected_set)}")
        print(f"  命中: {sorted(true_positives)}")
        print(f"  漏检: {sorted(false_negatives)}")
        print(f"  误检: {sorted(false_positives)}")
        print(f"  正确检测数: {true_positive_count}")
        print(f"  漏检比例: {miss_ratio:.2f}")
        print(f"  误检比例: {false_ratio:.2f}")
        print("-"*50)
    
    return {"expected_sum": expected_sum,
            "true_positive_sum": true_positive_sum,
            "detected_sum": detected_sum}

if __name__ == "__main__":
    # 指定包含 omi- 开头 txt 文件的文件夹（仅处理 omi- 文件）
    label_folder = r"data\interpretation_label"
    txt_files = [f for f in os.listdir(label_folder)
                 if f.startswith("omi-") and f.endswith(".txt")]
    
    global_expected = 0
    global_true_positive = 0
    global_detected = 0
    
    # 遍历每个 omi- 文件，根据文件名前缀加载对应的测试数据和测试标签文件
    for txt_filename in sorted(txt_files):
        prefix = txt_filename.split(".")[0]  # 例如 "omi-1"
        test_data_path = os.path.join("data/processed", f"{prefix}_test.pkl")
        test_labels_path = os.path.join("data/processed", f"{prefix}_test_label.pkl")
        
        if not (os.path.exists(test_data_path) and os.path.exists(test_labels_path)):
            print(f"缺少对应的测试数据或标签文件：{prefix}")
            continue
        
        test_data = pd.read_pickle(test_data_path)
        test_labels = pd.read_pickle(test_labels_path)
        if isinstance(test_data, np.ndarray):
            test_data = pd.DataFrame(test_data)
        if isinstance(test_labels, np.ndarray):
            test_labels = pd.DataFrame(test_labels, columns=["label"])
        elif isinstance(test_labels, pd.Series):
            test_labels = test_labels.to_frame(name="label")
        
        file_path = os.path.join(label_folder, txt_filename)
        file_stats = process_omi_file(file_path, test_data, test_labels)
        global_expected += file_stats["expected_sum"]
        global_true_positive += file_stats["true_positive_sum"]
        global_detected += file_stats["detected_sum"]
    
    # 计算总体统计
    recall = (global_true_positive / global_expected) if global_expected > 0 else 0
    precision = (global_true_positive / global_detected) if global_detected > 0 else 0
    
    print("="*30)
    print("总体统计:")
    print(f"总正确检测: {global_true_positive}/{global_expected}")
    print(f"召回率: {recall*100:.1f}%")
    print(f"精确率: {precision*100:.1f}%")
    print("="*30)


Processing file: data\interpretation_label\omi-1.txt
时段 760-765:
  期望异常传感器: [4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
  检测到异常传感器: [0, 3, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
  命中: [4, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
  漏检: [5, 19]
  误检: [0, 3, 6]
  正确检测数: 13
  漏检比例: 0.13
  误检比例: 0.19
--------------------------------------------------
时段 1064-1298:
  期望异常传感器: [6, 7, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
  检测到异常传感器: [0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
  命中: [6, 7, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
  漏检: [19]
  误检: [0, 2, 3, 4, 5, 8]
  正确检测数: 12
  漏检比例: 0.08
  误检比例: 0.33
--------------------------------------------------
时段 2758-2772:
  期望异常传感器: [5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
  检测到异常传感器: [0, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
  命中: [5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
  漏检: [19]
  误检: [0, 3, 4]
  正确检测数: 14
  漏检比例: 0.07
  误检比例: 0.18
-------------------

In [45]:
# 实验结果总体统计:
#   总期望异常传感器数: 658
#   总正确检测数: 554
#   总检测到异常传感器数: 1224
#   全局召回率: 84.2%
#   全局精确率: 45.3%

import os
import numpy as np
from scipy.stats import pearsonr
from dtaidistance import dtw
import networkx as nx
from gensim.models import Word2Vec
from sklearn.cluster import KMeans
import pandas as pd

def robust_zscore(x, median_val, mad):
    return np.abs(x - median_val) / (mad + 1e-8)

def strict_image_method(data, labels, window_size=5):
    """
    改进后的异常检测方法：
    1. 构建图结构：节点为连续时间窗口（窗口内数据存储在节点属性 'values'），
       边的条件：窗口展平后的 Pearson 相关系数绝对值 > 0.65，
       且各窗口沿时间方向均值序列的 DTW 距离 < 1.6。
    2. 利用随机游走和 Word2Vec（DeepWalk 思想）获得每个窗口的低维嵌入表示，
       随机游走次数增加到20次，每个游走步长延长到20。
    3. 用 KMeans 聚类（聚类数设为2，数量较少的簇视为异常窗口），
       将异常窗口映射回原始时间点，再对真实异常点（labels==1）利用鲁棒统计（中位数和 MAD）计算局部异常得分，
       返回异常传感器索引（得分大于0）。
    """
    # 1. 构建图结构
    G = nx.Graph()
    n_windows = len(data) - window_size + 1
    if n_windows < 2:
        return np.array([])
    for i in range(n_windows):
        window = data[i:i+window_size]
        G.add_node(i, values=window)
    
    # 添加边：仅比较窗口 i 与 i+1 到 i+20 内的窗口
    for i in range(n_windows):
        for j in range(i+1, min(i+20, n_windows)):
            corr = pearsonr(G.nodes[i]['values'].flatten(), 
                            G.nodes[j]['values'].flatten())[0]
            dtw_dist = dtw.distance(G.nodes[i]['values'].mean(axis=1),
                                    G.nodes[j]['values'].mean(axis=1))
            if abs(corr) > 0.65 and dtw_dist < 1.6:
                G.add_edge(i, j, weight=abs(corr))
    
    # 2. DeepWalk：随机游走获得节点序列，再用 Word2Vec 得到嵌入
    def random_walk(G, walk_length=20, num_walks=20):
        walks = []
        for _ in range(num_walks):
            for node in G.nodes():
                walk = [str(node)]
                while len(walk) < walk_length:
                    current = int(walk[-1])
                    neighbors = list(G.neighbors(current))
                    if neighbors:
                        walk.append(str(np.random.choice(neighbors)))
                    else:
                        break
                walks.append(walk)
        return walks

    walks = random_walk(G)
    model = Word2Vec(walks, vector_size=32, window=5, min_count=0, sg=1)
    embeddings = np.array([model.wv[str(i)] for i in G.nodes()])
    
    # 3. 异常检测：利用 KMeans 聚类，选择数量较少的簇作为异常窗口
    if embeddings.shape[0] < 2:
        return np.array([])
    kmeans = KMeans(n_clusters=2, random_state=42)
    clusters = kmeans.fit_predict(embeddings)
    anomalies = np.where(clusters == np.argmin(np.bincount(clusters)))[0]
    
    # 将异常窗口映射回原始时间点
    anomaly_windows = [i for i in anomalies if i in G.nodes()]
    anomaly_points = []
    for win in anomaly_windows:
        anomaly_points.extend(range(win, win+window_size))
    
    # 针对每个异常点（仅统计真实异常点 labels==1），使用鲁棒统计计算局部异常得分
    sensor_scores = np.zeros(data.shape[1])
    for point in set(anomaly_points):
        if point < len(labels) and labels[point] == 1:
            for sensor in range(data.shape[1]):
                local_data = data[max(0, point-100):point+1, sensor]
                median_val = np.median(local_data)
                mad = np.median(np.abs(local_data - median_val))
                sensor_scores[sensor] += robust_zscore(data[point, sensor], median_val, mad)
    return np.where(sensor_scores > 0)[0]

def process_omi_file(txt_file_path, test_data, test_labels):
    """
    处理单个 omi- 文件：
      1. 解析每行记录（格式：start-end:expected_sensor_list）；
      2. 对每个时段从 test_data 和 test_labels 中提取数据，调用 strict_image_method 进行检测；
      3. 输出每个时段的检测结果及对比，同时返回该文件所有时段的统计数据。
    返回一个字典，包含：
      - expected_sum: 期望异常传感器总数
      - true_positive_sum: 命中总数
      - detected_sum: 检测到的异常传感器总数
    """
    print("="*60)
    print("Processing file:", txt_file_path)
    with open(txt_file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    
    expected_sum = 0
    true_positive_sum = 0
    detected_sum = 0
    
    for line in lines:
        line = line.strip()
        if not line:
            continue
        if ':' not in line:
            print("格式错误，缺少冒号：", line)
            continue
        time_part, sensor_part = line.split(":", 1)
        if '-' not in time_part:
            print("格式错误，缺少 '-'：", line)
            continue
        try:
            start_str, end_str = time_part.split('-')
            start_idx = int(start_str)
            end_idx = int(end_str)
        except ValueError:
            print("时段解析错误：", line)
            continue
        if sensor_part.strip():
            try:
                expected_sensors = [int(x) for x in sensor_part.split(',') if x.strip() != '']
            except ValueError:
                print("传感器列表解析错误：", line)
                expected_sensors = []
        else:
            expected_sensors = []
        
        # 提取该时段数据（转换为 numpy 数组）
        seg_data = test_data.iloc[start_idx:end_idx+1].to_numpy()
        seg_labels = test_labels.iloc[start_idx:end_idx+1]["label"].to_numpy()
        
        detected_sensors = strict_image_method(seg_data, seg_labels)
        
        detected_set = set(detected_sensors)
        expected_set = set(expected_sensors)
        true_positives = detected_set & expected_set
        false_negatives = expected_set - detected_set
        false_positives = detected_set - expected_set
        
        expected_count = len(expected_set)
        true_positive_count = len(true_positives)
        detected_count = len(detected_set)
        
        expected_sum += expected_count
        true_positive_sum += true_positive_count
        detected_sum += detected_count
        
        miss_ratio = len(false_negatives) / expected_count if expected_count > 0 else 0
        false_ratio = len(false_positives) / detected_count if detected_count > 0 else 0
        
        print(f"时段 {start_idx}-{end_idx}:")
        print(f"  期望异常传感器: {sorted(expected_set)}")
        print(f"  检测到异常传感器: {sorted(detected_set)}")
        print(f"  命中: {sorted(true_positives)}")
        print(f"  漏检: {sorted(false_negatives)}")
        print(f"  误检: {sorted(false_positives)}")
        print(f"  正确检测数: {true_positive_count}")
        print(f"  漏检比例: {miss_ratio:.2f}")
        print(f"  误检比例: {false_ratio:.2f}")
        print("-"*50)
    
    # 计算当前文件的召回率和精确率
    recall = (true_positive_sum / expected_sum) if expected_sum > 0 else 0
    precision = (true_positive_sum / detected_sum) if detected_sum > 0 else 0
    
    print("当前文件统计:")
    print(f"  总期望异常传感器数: {expected_sum}")
    print(f"  总正确检测数: {true_positive_sum}")
    print(f"  总检测到异常传感器数: {detected_sum}")
    print(f"  召回率: {recall*100:.1f}%")
    print(f"  精确率: {precision*100:.1f}%")
    print("="*60)
    
    return {"expected_sum": expected_sum,
            "true_positive_sum": true_positive_sum,
            "detected_sum": detected_sum}

if __name__ == "__main__":
    # 指定包含 omi- 开头 txt 文件的文件夹（只处理 omi- 文件）
    label_folder = r"data\interpretation_label"
    txt_files = [f for f in os.listdir(label_folder)
                 if f.startswith("omi-") and f.endswith(".txt")]
    
    global_expected = 0
    global_true_positive = 0
    global_detected = 0
    
    for txt_filename in sorted(txt_files):
        prefix = txt_filename.split(".")[0]  # 例如 "omi-1"
        test_data_path = os.path.join("data/processed", f"{prefix}_test.pkl")
        test_labels_path = os.path.join("data/processed", f"{prefix}_test_label.pkl")
        
        if not (os.path.exists(test_data_path) and os.path.exists(test_labels_path)):
            print(f"缺少对应的测试数据或标签文件：{prefix}")
            continue
        
        test_data = pd.read_pickle(test_data_path)
        test_labels = pd.read_pickle(test_labels_path)
        if isinstance(test_data, np.ndarray):
            test_data = pd.DataFrame(test_data)
        if isinstance(test_labels, np.ndarray):
            test_labels = pd.DataFrame(test_labels, columns=["label"])
        elif isinstance(test_labels, pd.Series):
            test_labels = test_labels.to_frame(name="label")
        
        file_path = os.path.join(label_folder, txt_filename)
        file_stats = process_omi_file(file_path, test_data, test_labels)
        
        global_expected += file_stats["expected_sum"]
        global_true_positive += file_stats["true_positive_sum"]
        global_detected += file_stats["detected_sum"]
    
    # 全局统计
    global_recall = (global_true_positive / global_expected) if global_expected > 0 else 0
    global_precision = (global_true_positive / global_detected) if global_detected > 0 else 0
    
    print("总体统计:")
    print(f"  总期望异常传感器数: {global_expected}")
    print(f"  总正确检测数: {global_true_positive}")
    print(f"  总检测到异常传感器数: {global_detected}")
    print(f"  全局召回率: {global_recall*100:.1f}%")
    print(f"  全局精确率: {global_precision*100:.1f}%")


Processing file: data\interpretation_label\omi-1.txt
时段 760-765:
  期望异常传感器: [4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
  检测到异常传感器: [0, 3, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
  命中: [4, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
  漏检: [5, 19]
  误检: [0, 3, 6]
  正确检测数: 13
  漏检比例: 0.13
  误检比例: 0.19
--------------------------------------------------
时段 1064-1298:
  期望异常传感器: [6, 7, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
  检测到异常传感器: [0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
  命中: [6, 7, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
  漏检: [19]
  误检: [0, 2, 3, 4, 5, 8]
  正确检测数: 12
  漏检比例: 0.08
  误检比例: 0.33
--------------------------------------------------
时段 2758-2772:
  期望异常传感器: [5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
  检测到异常传感器: [0, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
  命中: [5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
  漏检: [19]
  误检: [0, 3, 4]
  正确检测数: 14
  漏检比例: 0.07
  误检比例: 0.18
-------------------

In [47]:
import os
import numpy as np
from scipy.stats import pearsonr
from dtaidistance import dtw
import networkx as nx
from gensim.models import Word2Vec
from sklearn.cluster import KMeans
import pandas as pd

def robust_zscore(x, median_val, mad):
    return np.abs(x - median_val) / (mad + 1e-8)

def strict_image_method(data, labels, window_size=5, score_threshold=0.5):
    """
    改进后的异常检测方法：
    1. 构建图结构：节点为连续时间窗口（窗口内数据存储在节点属性 'values'），
       边的条件：窗口展平后的 Pearson 相关系数绝对值 > 0.65，
       且各窗口沿时间方向均值序列的 DTW 距离 < 1.6。
    2. 利用随机游走和 Word2Vec（DeepWalk 思想）获得每个窗口的低维嵌入表示，
       随机游走次数设为20次，游走步长延长到20。
    3. 用 KMeans 聚类（聚类数设为2，数量较少的簇视为异常窗口），
       将异常窗口映射回原始时间点，再对真实异常点（labels==1）利用鲁棒统计（中位数和 MAD）计算局部异常得分。
       最后，对每个传感器计算参与异常点的平均得分，当平均得分超过 score_threshold 时，认为该传感器异常。
    """
    # 1. 构建图结构
    G = nx.Graph()
    n_windows = len(data) - window_size + 1
    if n_windows < 2:
        return np.array([])
    for i in range(n_windows):
        window = data[i:i+window_size]
        G.add_node(i, values=window)
    
    # 添加边：比较窗口 i 与 i+1 到 i+20 内的窗口
    for i in range(n_windows):
        for j in range(i+1, min(i+20, n_windows)):
            corr = pearsonr(G.nodes[i]['values'].flatten(), 
                            G.nodes[j]['values'].flatten())[0]
            dtw_dist = dtw.distance(G.nodes[i]['values'].mean(axis=1),
                                    G.nodes[j]['values'].mean(axis=1))
            if abs(corr) > 0.65 and dtw_dist < 1.6:
                G.add_edge(i, j, weight=abs(corr))
    
    # 2. DeepWalk：随机游走获得节点序列，再用 Word2Vec 得到嵌入
    def random_walk(G, walk_length=20, num_walks=20):
        walks = []
        for _ in range(num_walks):
            for node in G.nodes():
                walk = [str(node)]
                while len(walk) < walk_length:
                    current = int(walk[-1])
                    neighbors = list(G.neighbors(current))
                    if neighbors:
                        walk.append(str(np.random.choice(neighbors)))
                    else:
                        break
                walks.append(walk)
        return walks

    walks = random_walk(G)
    model = Word2Vec(walks, vector_size=32, window=5, min_count=0, sg=1)
    embeddings = np.array([model.wv[str(i)] for i in G.nodes()])
    
    # 3. 异常检测：利用 KMeans 聚类，选出数量较少的簇作为异常窗口
    if embeddings.shape[0] < 2:
        return np.array([])
    kmeans = KMeans(n_clusters=2, random_state=42)
    clusters = kmeans.fit_predict(embeddings)
    anomalies = np.where(clusters == np.argmin(np.bincount(clusters)))[0]
    
    # 将异常窗口映射回原始时间点
    anomaly_windows = [i for i in anomalies if i in G.nodes()]
    anomaly_points = []
    for win in anomaly_windows:
        anomaly_points.extend(range(win, win+window_size))
    
    # 针对每个异常点（仅统计真实异常点 labels==1），计算每个传感器的局部异常得分
    sensor_scores = np.zeros(data.shape[1])
    sensor_counts = np.zeros(data.shape[1])
    for point in set(anomaly_points):
        if point < len(labels) and labels[point] == 1:
            for sensor in range(data.shape[1]):
                local_data = data[max(0, point-100):point+1, sensor]
                median_val = np.median(local_data)
                mad = np.median(np.abs(local_data - median_val))
                score = robust_zscore(data[point, sensor], median_val, mad)
                sensor_scores[sensor] += score
                sensor_counts[sensor] += 1
    # 计算每个传感器的平均得分
    avg_scores = np.zeros(data.shape[1])
    for sensor in range(data.shape[1]):
        if sensor_counts[sensor] > 0:
            avg_scores[sensor] = sensor_scores[sensor] / sensor_counts[sensor]
    
    # 返回平均得分大于 score_threshold 的传感器索引
    return np.where(avg_scores > score_threshold)[0]

def process_omi_file(txt_file_path, test_data, test_labels):
    """
    处理单个 omi- 文件：
      1. 解析每行记录（格式：start-end:expected_sensor_list）；
      2. 对每个时段从 test_data 和 test_labels 中提取数据，调用 strict_image_method 进行检测；
      3. 输出每个时段的检测结果及对比，同时返回该文件所有时段的统计数据。
    返回一个字典，包含：
      - expected_sum: 期望异常传感器总数
      - true_positive_sum: 命中总数
      - detected_sum: 检测到的异常传感器总数
    """
    print("="*60)
    print("Processing file:", txt_file_path)
    with open(txt_file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    
    expected_sum = 0
    true_positive_sum = 0
    detected_sum = 0
    
    for line in lines:
        line = line.strip()
        if not line:
            continue
        if ':' not in line:
            print("格式错误，缺少冒号：", line)
            continue
        time_part, sensor_part = line.split(":", 1)
        if '-' not in time_part:
            print("格式错误，缺少 '-'：", line)
            continue
        try:
            start_str, end_str = time_part.split('-')
            start_idx = int(start_str)
            end_idx = int(end_str)
        except ValueError:
            print("时段解析错误：", line)
            continue
        if sensor_part.strip():
            try:
                expected_sensors = [int(x) for x in sensor_part.split(',') if x.strip() != '']
            except ValueError:
                print("传感器列表解析错误：", line)
                expected_sensors = []
        else:
            expected_sensors = []
        
        # 提取该时段数据（转换为 numpy 数组）
        seg_data = test_data.iloc[start_idx:end_idx+1].to_numpy()
        seg_labels = test_labels.iloc[start_idx:end_idx+1]["label"].to_numpy()
        
        detected_sensors = strict_image_method(seg_data, seg_labels)
        
        detected_set = set(detected_sensors)
        expected_set = set(expected_sensors)
        true_positives = detected_set & expected_set
        false_negatives = expected_set - detected_set
        false_positives = detected_set - expected_set
        
        expected_count = len(expected_set)
        true_positive_count = len(true_positives)
        detected_count = len(detected_set)
        
        expected_sum += expected_count
        true_positive_sum += true_positive_count
        detected_sum += detected_count
        
        miss_ratio = len(false_negatives) / expected_count if expected_count > 0 else 0
        false_ratio = len(false_positives) / detected_count if detected_count > 0 else 0
        
        print(f"时段 {start_idx}-{end_idx}:")
        print(f"  期望异常传感器: {sorted(expected_set)}")
        print(f"  检测到异常传感器: {sorted(detected_set)}")
        print(f"  命中: {sorted(true_positives)}")
        print(f"  漏检: {sorted(false_negatives)}")
        print(f"  误检: {sorted(false_positives)}")
        print(f"  正确检测数: {true_positive_count}")
        print(f"  漏检比例: {miss_ratio:.2f}")
        print(f"  误检比例: {false_ratio:.2f}")
        print("-"*50)
    
    # 计算当前文件的召回率和精确率
    recall = (true_positive_sum / expected_sum) if expected_sum > 0 else 0
    precision = (true_positive_sum / detected_sum) if detected_sum > 0 else 0
    
    print("当前文件统计:")
    print(f"  总期望异常传感器数: {expected_sum}")
    print(f"  总正确检测数: {true_positive_sum}")
    print(f"  总检测到异常传感器数: {detected_sum}")
    print(f"  召回率: {recall*100:.1f}%")
    print(f"  精确率: {precision*100:.1f}%")
    print("="*60)
    
    return {"expected_sum": expected_sum,
            "true_positive_sum": true_positive_sum,
            "detected_sum": detected_sum}

if __name__ == "__main__":
    # 指定包含 omi- 开头 txt 文件的文件夹（仅处理 omi- 文件）
    label_folder = r"data\interpretation_label"
    txt_files = [f for f in os.listdir(label_folder)
                 if f.startswith("omi-") and f.endswith(".txt")]
    
    global_expected = 0
    global_true_positive = 0
    global_detected = 0
    
    for txt_filename in sorted(txt_files):
        prefix = txt_filename.split(".")[0]  # 例如 "omi-1"
        test_data_path = os.path.join("data/processed", f"{prefix}_test.pkl")
        test_labels_path = os.path.join("data/processed", f"{prefix}_test_label.pkl")
        
        if not (os.path.exists(test_data_path) and os.path.exists(test_labels_path)):
            print(f"缺少对应的测试数据或标签文件：{prefix}")
            continue
        
        test_data = pd.read_pickle(test_data_path)
        test_labels = pd.read_pickle(test_labels_path)
        if isinstance(test_data, np.ndarray):
            test_data = pd.DataFrame(test_data)
        if isinstance(test_labels, np.ndarray):
            test_labels = pd.DataFrame(test_labels, columns=["label"])
        elif isinstance(test_labels, pd.Series):
            test_labels = test_labels.to_frame(name="label")
        
        file_path = os.path.join(label_folder, txt_filename)
        file_stats = process_omi_file(file_path, test_data, test_labels)
        
        global_expected += file_stats["expected_sum"]
        global_true_positive += file_stats["true_positive_sum"]
        global_detected += file_stats["detected_sum"]
        
        # 输出当前文件后的全局统计
        current_recall = (global_true_positive / global_expected) if global_expected > 0 else 0
        current_precision = (global_true_positive / global_detected) if global_detected > 0 else 0
        print("当前累计总体统计:")
        print(f"  总期望异常传感器数: {global_expected}")
        print(f"  总正确检测数: {global_true_positive}")
        print(f"  总检测到异常传感器数: {global_detected}")
        print(f"  累计召回率: {current_recall*100:.1f}%")
        print(f"  累计精确率: {current_precision*100:.1f}%")
        print("="*60)
    
    # 最后输出全局总体统计
    global_recall = (global_true_positive / global_expected) if global_expected > 0 else 0
    global_precision = (global_true_positive / global_detected) if global_detected > 0 else 0
    
    print("最终总体统计:")
    print(f"  总期望异常传感器数: {global_expected}")
    print(f"  总正确检测数: {global_true_positive}")
    print(f"  总检测到异常传感器数: {global_detected}")
    print(f"  全局召回率: {global_recall*100:.1f}%")
    print(f"  全局精确率: {global_precision*100:.1f}%")


Processing file: data\interpretation_label\omi-1.txt
时段 760-765:
  期望异常传感器: [4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
  检测到异常传感器: [0, 4, 6, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
  命中: [4, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
  漏检: [5, 7, 19]
  误检: [0, 6]
  正确检测数: 12
  漏检比例: 0.20
  误检比例: 0.14
--------------------------------------------------
时段 1064-1298:
  期望异常传感器: [6, 7, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
  检测到异常传感器: [0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
  命中: [6, 7, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
  漏检: [19]
  误检: [0, 2, 3, 4, 5, 8]
  正确检测数: 12
  漏检比例: 0.08
  误检比例: 0.33
--------------------------------------------------
时段 2758-2772:
  期望异常传感器: [5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
  检测到异常传感器: [0, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
  命中: [5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
  漏检: [19]
  误检: [0, 3, 4]
  正确检测数: 14
  漏检比例: 0.07
  误检比例: 0.18
----------------------------

In [48]:
import os
import numpy as np
from scipy.stats import pearsonr
from dtaidistance import dtw
import networkx as nx
from gensim.models import Word2Vec
from sklearn.cluster import KMeans
import pandas as pd
from sklearn.metrics import f1_score

def robust_zscore(x, median_val, mad):
    return np.abs(x - median_val) / (mad + 1e-8)

def dynamic_threshold(data_segment):
    """动态调整相关系数阈值"""
    if len(data_segment) < 2:
        return 0.65
    corrs = [pearsonr(data_segment[i].flatten(), data_segment[i+1].flatten())[0] 
             for i in range(len(data_segment)-1)]
    median_corr = np.median(corrs)
    return max(0.65, median_corr - 0.07)  # 最低保持0.65，根据数据动态调整

def strict_image_method(data, labels, window_size=5):
    # 1. 构建图结构
    G = nx.Graph()
    n_windows = len(data) - window_size + 1
    if n_windows < 2:
        return np.array([])
    
    # 存储所有窗口数据用于动态阈值计算
    window_data = []
    for i in range(n_windows):
        window = data[i:i+window_size]
        G.add_node(i, values=window)
        window_data.append(window)
    
    # 动态获取阈值
    corr_threshold = dynamic_threshold(window_data)
    dtw_threshold = 1.4  # 平衡值
    
    # 添加边：比较窗口i与i+1到i+15内的窗口
    for i in range(n_windows):
        for j in range(i+1, min(i+15, n_windows)):
            corr = pearsonr(G.nodes[i]['values'].flatten(), 
                          G.nodes[j]['values'].flatten())[0]
            dtw_dist = dtw.distance(G.nodes[i]['values'].mean(axis=1),
                                  G.nodes[j]['values'].mean(axis=1))
            if abs(corr) > corr_threshold and dtw_dist < dtw_threshold:
                G.add_edge(i, j, weight=abs(corr))
    
    # 2. DeepWalk
    def random_walk(G, walk_length=15, num_walks=15):
        walks = []
        for _ in range(num_walks):
            for node in G.nodes():
                walk = [str(node)]
                while len(walk) < walk_length:
                    current = int(walk[-1])
                    neighbors = list(G.neighbors(current))
                    if neighbors:
                        walk.append(str(np.random.choice(neighbors)))
                    else:
                        break
                walks.append(walk)
        return walks

    walks = random_walk(G)
    model = Word2Vec(walks, vector_size=32, window=5, min_count=0, sg=1)
    embeddings = np.array([model.wv[str(i)] for i in G.nodes()])
    
    # 3. 异常检测
    if embeddings.shape[0] < 2:
        return np.array([])
    
    # 使用KMeans++初始化
    kmeans = KMeans(n_clusters=2, init='k-means++', random_state=42)
    clusters = kmeans.fit_predict(embeddings)
    
    # 确定异常簇（考虑簇大小和平均异常分数）
    anomaly_cluster = np.argmin(np.bincount(clusters))
    anomalies = np.where(clusters == anomaly_cluster)[0]
    
    # 异常确认机制：要求异常窗口被多次检测到
    confirmed_anomalies = []
    for win in anomalies:
        if win in G.nodes():
            # 检查该窗口是否被多个相邻窗口也标记为异常
            neighbor_anomalies = sum(1 for j in range(max(0,win-2), min(win+3, n_windows))
                                   if j in anomalies)
            if neighbor_anomalies >= 2:  # 至少被2个相邻窗口确认
                confirmed_anomalies.append(win)
    
    # 映射回原始时间点
    anomaly_points = []
    for win in confirmed_anomalies:
        anomaly_points.extend(range(win, win+window_size))
    
    # 计算传感器异常分数（使用更严格的局部窗口）
    sensor_scores = np.zeros(data.shape[1])
    for point in set(anomaly_points):
        if point < len(labels) and labels[point] == 1:
            for sensor in range(data.shape[1]):
                local_data = data[max(0, point-50):point+1, sensor]  # 缩小局部窗口
                median_val = np.median(local_data)
                mad = np.median(np.abs(local_data - median_val))
                score = robust_zscore(data[point, sensor], median_val, mad)
                if score > 0.5:  # 平衡阈值
                    sensor_scores[sensor] += score
    
    return np.where(sensor_scores > 0)[0]

def process_omi_file(txt_file_path, test_data, test_labels):
    print("="*60)
    print("Processing file:", txt_file_path)
    
    with open(txt_file_path, 'r', encoding='utf-8') as f:
        lines = [line.strip() for line in f.readlines() if line.strip()]
    
    expected_sum = 0
    true_positive_sum = 0
    detected_sum = 0
    
    for line in lines:
        if ':' not in line:
            continue
            
        time_part, sensor_part = line.split(":", 1)
        if '-' not in time_part:
            continue
            
        try:
            start_idx, end_idx = map(int, time_part.split('-'))
        except ValueError:
            continue
            
        expected_sensors = []
        if sensor_part.strip():
            try:
                expected_sensors = list(map(int, sensor_part.split(',')))
            except ValueError:
                continue
        
        # 提取数据
        seg_data = test_data.iloc[start_idx:end_idx+1].to_numpy()
        seg_labels = test_labels.iloc[start_idx:end_idx+1]["label"].to_numpy()
        
        detected_sensors = strict_image_method(seg_data, seg_labels)
        
        # 计算指标
        detected_set = set(detected_sensors)
        expected_set = set(expected_sensors)
        true_positives = detected_set & expected_set
        false_negatives = expected_set - detected_set
        false_positives = detected_set - expected_set
        
        expected_count = len(expected_set)
        true_positive_count = len(true_positives)
        detected_count = len(detected_set)
        
        expected_sum += expected_count
        true_positive_sum += true_positive_count
        detected_sum += detected_count
        
        # 打印结果
        print(f"时段 {start_idx}-{end_idx}:")
        print(f"  期望异常传感器: {sorted(expected_set)}")
        print(f"  检测到异常传感器: {sorted(detected_set)}")
        print(f"  命中: {sorted(true_positives)}")
        print(f"  漏检: {sorted(false_negatives)}")
        print(f"  误检: {sorted(false_positives)}")
        print(f"  正确检测数: {true_positive_count}")
        print("-"*50)
    
    # 计算评估指标
    recall = true_positive_sum / expected_sum if expected_sum > 0 else 0
    precision = true_positive_sum / detected_sum if detected_sum > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    print("当前文件统计:")
    print(f"  总期望异常传感器数: {expected_sum}")
    print(f"  总正确检测数: {true_positive_sum}")
    print(f"  总检测到异常传感器数: {detected_sum}")
    print(f"  召回率: {recall*100:.1f}%")
    print(f"  精确率: {precision*100:.1f}%")
    print(f"  F1分数: {f1 * 100:.1f}%")
    print("="*60)
    
    return {
        "expected_sum": expected_sum,
        "true_positive_sum": true_positive_sum,
        "detected_sum": detected_sum
    }

if __name__ == "__main__":
    label_folder = r"data\interpretation_label"
    txt_files = sorted([f for f in os.listdir(label_folder) 
                       if f.startswith("omi-") and f.endswith(".txt")])
    
    global_stats = {
        "expected_sum": 0,
        "true_positive_sum": 0,
        "detected_sum": 0
    }
    
    for txt_filename in txt_files:
        prefix = txt_filename.split(".")[0]
        test_data_path = os.path.join("data/processed", f"{prefix}_test.pkl")
        test_labels_path = os.path.join("data/processed", f"{prefix}_test_label.pkl")
        
        if not (os.path.exists(test_data_path) and os.path.exists(test_labels_path)):
            continue
            
        test_data = pd.read_pickle(test_data_path)
        test_labels = pd.read_pickle(test_labels_path)
        
        # 统一数据格式
        if isinstance(test_data, np.ndarray):
            test_data = pd.DataFrame(test_data)
        if isinstance(test_labels, np.ndarray):
            test_labels = pd.DataFrame(test_labels, columns=["label"])
        elif isinstance(test_labels, pd.Series):
            test_labels = test_labels.to_frame(name="label")
        
        file_path = os.path.join(label_folder, txt_filename)
        file_stats = process_omi_file(file_path, test_data, test_labels)
        
        for key in global_stats:
            global_stats[key] += file_stats[key]
    
    # 全局统计
    recall = (global_stats["true_positive_sum"] / global_stats["expected_sum"] 
              if global_stats["expected_sum"] > 0 else 0)
    precision = (global_stats["true_positive_sum"] / global_stats["detected_sum"] 
                if global_stats["detected_sum"] > 0 else 0)
    f1 = (2 * (precision * recall) / (precision + recall) 
          if (precision + recall) > 0 else 0)
    
    print("\n总体统计:")
    print(f"  处理文件数: {len(txt_files)}")
    print(f"  总期望异常传感器数: {global_stats['expected_sum']}")
    print(f"  总正确检测数: {global_stats['true_positive_sum']}")
    print(f"  总检测到异常传感器数: {global_stats['detected_sum']}")
    print(f"  全局召回率: {recall*100:.1f}%")
    print(f"  全局精确率: {precision*100:.1f}%")
    print(f"  全局F1分数: {f1 * 100:.1f}%")

Processing file: data\interpretation_label\omi-1.txt
时段 760-765:
  期望异常传感器: [4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
  检测到异常传感器: []
  命中: []
  漏检: [4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
  误检: []
  正确检测数: 0
--------------------------------------------------
时段 1064-1298:
  期望异常传感器: [6, 7, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
  检测到异常传感器: [0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
  命中: [6, 7, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
  漏检: [19]
  误检: [0, 2, 3, 4, 5, 8]
  正确检测数: 12
--------------------------------------------------
时段 2758-2772:
  期望异常传感器: [5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
  检测到异常传感器: [0, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
  命中: [5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
  漏检: [19]
  误检: [0, 3, 4]
  正确检测数: 14
--------------------------------------------------
时段 2874-2885:
  期望异常传感器: [7, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
  检测到异常传感器: [0, 3, 4, 5, 6, 7, 8, 9, 10

In [50]:
import os
import numpy as np
from scipy.stats import pearsonr
from dtaidistance import dtw
import networkx as nx
from gensim.models import Word2Vec
from sklearn.cluster import DBSCAN
import pandas as pd

def robust_zscore(x, median_val, mad):
    return np.abs(x - median_val) / (mad + 1e-8)

def strict_image_method(data, labels, window_size=5, score_threshold=0.5):
    """
    使用 DBSCAN 替代 KMeans 进行嵌入聚类，实现异常窗口识别与传感器定位。
    """
    G = nx.Graph()
    n_windows = len(data) - window_size + 1
    if n_windows < 2:
        return np.array([])
    
    for i in range(n_windows):
        window = data[i:i+window_size]
        G.add_node(i, values=window)

    for i in range(n_windows):
        for j in range(i+1, min(i+20, n_windows)):
            corr = pearsonr(G.nodes[i]['values'].flatten(), G.nodes[j]['values'].flatten())[0]
            dtw_dist = dtw.distance(G.nodes[i]['values'].mean(axis=1), G.nodes[j]['values'].mean(axis=1))
            if abs(corr) > 0.65 and dtw_dist < 1.6:
                G.add_edge(i, j, weight=abs(corr))

    # DeepWalk嵌入
    def random_walk(G, walk_length=20, num_walks=20):
        walks = []
        for _ in range(num_walks):
            for node in G.nodes():
                walk = [str(node)]
                while len(walk) < walk_length:
                    current = int(walk[-1])
                    neighbors = list(G.neighbors(current))
                    if neighbors:
                        walk.append(str(np.random.choice(neighbors)))
                    else:
                        break
                walks.append(walk)
        return walks

    walks = random_walk(G)
    model = Word2Vec(walks, vector_size=32, window=5, min_count=0, sg=1)
    embeddings = np.array([model.wv[str(i)] for i in G.nodes()])
    
    if embeddings.shape[0] < 2:
        return np.array([])

    # 🚀 使用 DBSCAN 替代 KMeans 聚类
    dbscan = DBSCAN(eps=0.6, min_samples=3)  # 可调参数
    clusters = dbscan.fit_predict(embeddings)

    # DBSCAN 中 -1 视为异常窗口
    anomalies = np.where(clusters == -1)[0]

    # 异常窗口 ➜ 异常时间点
    anomaly_points = []
    for win in anomalies:
        anomaly_points.extend(range(win, win+window_size))

    # 每个传感器的累计异常得分
    sensor_scores = np.zeros(data.shape[1])
    sensor_counts = np.zeros(data.shape[1])
    for point in set(anomaly_points):
        if point < len(labels) and labels[point] == 1:
            for sensor in range(data.shape[1]):
                local = data[max(0, point-100):point+1, sensor]
                median = np.median(local)
                mad = np.median(np.abs(local - median))
                score = robust_zscore(data[point, sensor], median, mad)
                sensor_scores[sensor] += score
                sensor_counts[sensor] += 1

    avg_scores = sensor_scores / (sensor_counts + 1e-8)
    return np.where(avg_scores > score_threshold)[0]


# 后续的 process_omi_file 和主函数不变，仍可与前述结构配套使用
def process_omi_file(txt_file_path, test_data, test_labels):
    print("="*60)
    print("Processing file:", txt_file_path)
    
    with open(txt_file_path, 'r', encoding='utf-8') as f:
        lines = [line.strip() for line in f.readlines() if line.strip()]
    
    expected_sum = 0
    true_positive_sum = 0
    detected_sum = 0
    
    for line in lines:
        if ':' not in line:
            continue
            
        time_part, sensor_part = line.split(":", 1)
        if '-' not in time_part:
            continue
            
        try:
            start_idx, end_idx = map(int, time_part.split('-'))
        except ValueError:
            continue
            
        expected_sensors = []
        if sensor_part.strip():
            try:
                expected_sensors = list(map(int, sensor_part.split(',')))
            except ValueError:
                continue
        
        # 提取数据
        seg_data = test_data.iloc[start_idx:end_idx+1].to_numpy()
        seg_labels = test_labels.iloc[start_idx:end_idx+1]["label"].to_numpy()
        
        detected_sensors = strict_image_method(seg_data, seg_labels)
        
        # 计算指标
        detected_set = set(detected_sensors)
        expected_set = set(expected_sensors)
        true_positives = detected_set & expected_set
        false_negatives = expected_set - detected_set
        false_positives = detected_set - expected_set
        
        expected_count = len(expected_set)
        true_positive_count = len(true_positives)
        detected_count = len(detected_set)
        
        expected_sum += expected_count
        true_positive_sum += true_positive_count
        detected_sum += detected_count
        
        # 打印结果
        print(f"时段 {start_idx}-{end_idx}:")
        print(f"  期望异常传感器: {sorted(expected_set)}")
        print(f"  检测到异常传感器: {sorted(detected_set)}")
        print(f"  命中: {sorted(true_positives)}")
        print(f"  漏检: {sorted(false_negatives)}")
        print(f"  误检: {sorted(false_positives)}")
        print(f"  正确检测数: {true_positive_count}")
        print("-"*50)
    
    # 计算评估指标
    recall = true_positive_sum / expected_sum if expected_sum > 0 else 0
    precision = true_positive_sum / detected_sum if detected_sum > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    print("当前文件统计:")
    print(f"  总期望异常传感器数: {expected_sum}")
    print(f"  总正确检测数: {true_positive_sum}")
    print(f"  总检测到异常传感器数: {detected_sum}")
    print(f"  召回率: {recall*100:.1f}%")
    print(f"  精确率: {precision*100:.1f}%")
    print(f"  F1分数: {f1 * 100:.1f}%")
    print("="*60)
    
    return {
        "expected_sum": expected_sum,
        "true_positive_sum": true_positive_sum,
        "detected_sum": detected_sum
    }

if __name__ == "__main__":
    label_folder = r"data\interpretation_label"
    txt_files = sorted([f for f in os.listdir(label_folder) 
                       if f.startswith("omi-") and f.endswith(".txt")])
    
    global_stats = {
        "expected_sum": 0,
        "true_positive_sum": 0,
        "detected_sum": 0
    }
    
    for txt_filename in txt_files:
        prefix = txt_filename.split(".")[0]
        test_data_path = os.path.join("data/processed", f"{prefix}_test.pkl")
        test_labels_path = os.path.join("data/processed", f"{prefix}_test_label.pkl")
        
        if not (os.path.exists(test_data_path) and os.path.exists(test_labels_path)):
            continue
            
        test_data = pd.read_pickle(test_data_path)
        test_labels = pd.read_pickle(test_labels_path)
        
        # 统一数据格式
        if isinstance(test_data, np.ndarray):
            test_data = pd.DataFrame(test_data)
        if isinstance(test_labels, np.ndarray):
            test_labels = pd.DataFrame(test_labels, columns=["label"])
        elif isinstance(test_labels, pd.Series):
            test_labels = test_labels.to_frame(name="label")
        
        file_path = os.path.join(label_folder, txt_filename)
        file_stats = process_omi_file(file_path, test_data, test_labels)
        
        for key in global_stats:
            global_stats[key] += file_stats[key]
    
    # 全局统计
    recall = (global_stats["true_positive_sum"] / global_stats["expected_sum"] 
              if global_stats["expected_sum"] > 0 else 0)
    precision = (global_stats["true_positive_sum"] / global_stats["detected_sum"] 
                if global_stats["detected_sum"] > 0 else 0)
    f1 = (2 * (precision * recall) / (precision + recall) 
          if (precision + recall) > 0 else 0)
    
    print("\n总体统计:")
    print(f"  处理文件数: {len(txt_files)}")
    print(f"  总期望异常传感器数: {global_stats['expected_sum']}")
    print(f"  总正确检测数: {global_stats['true_positive_sum']}")
    print(f"  总检测到异常传感器数: {global_stats['detected_sum']}")
    print(f"  全局召回率: {recall*100:.1f}%")
    print(f"  全局精确率: {precision*100:.1f}%")
    print(f"  全局F1分数: {f1 * 100:.1f}%")

Processing file: data\interpretation_label\omi-1.txt
时段 760-765:
  期望异常传感器: [4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
  检测到异常传感器: [0, 4, 6, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
  命中: [4, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
  漏检: [5, 7, 19]
  误检: [0, 6]
  正确检测数: 12
--------------------------------------------------
时段 1064-1298:
  期望异常传感器: [6, 7, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
  检测到异常传感器: [0, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
  命中: [6, 7, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
  漏检: [19]
  误检: [0, 3, 4, 5, 8]
  正确检测数: 12
--------------------------------------------------
时段 2758-2772:
  期望异常传感器: [5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
  检测到异常传感器: []
  命中: []
  漏检: [5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
  误检: []
  正确检测数: 0
--------------------------------------------------
时段 2874-2885:
  期望异常传感器: [7, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
  检测到异常传感器: [0, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 

In [52]:
'''
data\interpretation_label\omi
实验结果最终总体统计:
  总期望异常传感器数: 658
  总正确检测数: 550
  总检测到异常传感器数: 1192
  全局召回率: 83.6%
  全局精确率: 46.1%

===================================

data\interpretation_label\machine
最终总体统计:
  总期望异常传感器数: 1167
  总正确检测数: 765
  总检测到异常传感器数: 2764
  全局召回率: 65.6%
  全局精确率: 27.7%
'''

import os
import numpy as np
from scipy.stats import pearsonr
from dtaidistance import dtw
import networkx as nx
from gensim.models import Word2Vec
from sklearn.cluster import KMeans
import pandas as pd

def robust_zscore(x, median_val, mad):
    # 鲁棒 Z 分数：(x - median) / (MAD + 1e-8)
    return np.abs(x - median_val) / (mad + 1e-8)

def strict_image_method(data, labels, window_size=5, score_threshold=0.5):
    """
    改进后的异常检测方法：
    1. 构建图结构：节点为连续时间窗口（窗口内数据存储在节点属性 'values'），
       边的条件：窗口展平后的 Pearson 相关系数绝对值 > 0.65，
       且各窗口沿时间方向均值序列的 DTW 距离 < 1.6。
    2. 利用随机游走和 Word2Vec（DeepWalk 思想）获得每个窗口的低维嵌入表示，
       随机游走次数设为20次，游走步长延长到20。
    3. 用 KMeans 聚类（聚类数设为2，数量较少的簇视为异常窗口），
       将异常窗口映射回原始时间点，再对真实异常点（labels==1）利用鲁棒统计（中位数和 MAD）计算局部异常得分。
       最后，对每个传感器计算参与异常点的平均得分，当平均得分超过 score_threshold 时，认为该传感器异常。
    """
    # 1. 构建图结构
    G = nx.Graph()
    n_windows = len(data) - window_size + 1
    if n_windows < 2:
        return np.array([])
    for i in range(n_windows):
        window = data[i:i+window_size]
        G.add_node(i, values=window)
    
    # 添加边：比较窗口 i 与 i+1 到 i+20 内的窗口
    for i in range(n_windows):
        for j in range(i+1, min(i+20, n_windows)):
            corr = pearsonr(G.nodes[i]['values'].flatten(), 
                            G.nodes[j]['values'].flatten())[0]
            dtw_dist = dtw.distance(G.nodes[i]['values'].mean(axis=1),
                                    G.nodes[j]['values'].mean(axis=1))
            if abs(corr) > 0.65 and dtw_dist < 1.6:
                G.add_edge(i, j, weight=abs(corr))
    
    # 2. DeepWalk：随机游走获得节点序列，再用 Word2Vec 得到嵌入
    def random_walk(G, walk_length=20, num_walks=20):
        walks = []
        for _ in range(num_walks):
            for node in G.nodes():
                walk = [str(node)]
                while len(walk) < walk_length:
                    current = int(walk[-1])
                    neighbors = list(G.neighbors(current))
                    if neighbors:
                        walk.append(str(np.random.choice(neighbors)))
                    else:
                        break
                walks.append(walk)
        return walks

    walks = random_walk(G)
    model = Word2Vec(walks, vector_size=32, window=5, min_count=0, sg=1)
    embeddings = np.array([model.wv[str(i)] for i in G.nodes()])
    
    # 3. 异常检测：利用 KMeans 聚类，选出数量较少的簇作为异常窗口
    if embeddings.shape[0] < 2:
        return np.array([])
    kmeans = KMeans(n_clusters=2, random_state=42)
    clusters = kmeans.fit_predict(embeddings)
    anomalies = np.where(clusters == np.argmin(np.bincount(clusters)))[0]
    
    # 将异常窗口映射回原始时间点
    anomaly_windows = [i for i in anomalies if i in G.nodes()]
    anomaly_points = []
    for win in anomaly_windows:
        anomaly_points.extend(range(win, win+window_size))
    
    # 针对每个异常点（仅统计真实异常点 labels==1），计算每个传感器的局部异常得分
    sensor_scores = np.zeros(data.shape[1])
    sensor_counts = np.zeros(data.shape[1])
    for point in set(anomaly_points):
        if point < len(labels) and labels[point] == 1:
            for sensor in range(data.shape[1]):
                local_data = data[max(0, point-100):point+1, sensor]
                median_val = np.median(local_data)
                mad = np.median(np.abs(local_data - median_val))
                score = robust_zscore(data[point, sensor], median_val, mad)
                sensor_scores[sensor] += score
                sensor_counts[sensor] += 1
    # 计算每个传感器的平均得分
    avg_scores = np.zeros(data.shape[1])
    for sensor in range(data.shape[1]):
        if sensor_counts[sensor] > 0:
            avg_scores[sensor] = sensor_scores[sensor] / sensor_counts[sensor]
    
    # 返回平均得分大于 score_threshold 的传感器索引
    return np.where(avg_scores > score_threshold)[0]

def process_omi_file(txt_file_path, test_data, test_labels):
    """
    处理单个 omi- 文件：
      1. 解析每行记录（格式：start-end:expected_sensor_list）；
      2. 对每个时段从 test_data 和 test_labels 中提取数据，调用 strict_image_method 进行检测；
      3. 输出每个时段的检测结果及对比，同时返回该文件所有时段的统计数据。
    返回一个字典，包含：
      - expected_sum: 期望异常传感器总数
      - true_positive_sum: 命中总数
      - detected_sum: 检测到的异常传感器总数
    """
    print("="*60)
    print("Processing file:", txt_file_path)
    with open(txt_file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    
    expected_sum = 0
    true_positive_sum = 0
    detected_sum = 0
    
    for line in lines:
        line = line.strip()
        if not line:
            continue
        if ':' not in line:
            print("格式错误，缺少冒号：", line)
            continue
        time_part, sensor_part = line.split(":", 1)
        if '-' not in time_part:
            print("格式错误，缺少 '-'：", line)
            continue
        try:
            start_str, end_str = time_part.split('-')
            start_idx = int(start_str)
            end_idx = int(end_str)
        except ValueError:
            print("时段解析错误：", line)
            continue
        if sensor_part.strip():
            try:
                expected_sensors = [int(x) for x in sensor_part.split(',') if x.strip() != '']
            except ValueError:
                print("传感器列表解析错误：", line)
                expected_sensors = []
        else:
            expected_sensors = []
        
        # 提取该时段数据（转换为 numpy 数组）
        seg_data = test_data.iloc[start_idx:end_idx+1].to_numpy()
        seg_labels = test_labels.iloc[start_idx:end_idx+1]["label"].to_numpy()
        
        detected_sensors = strict_image_method(seg_data, seg_labels)
        
        detected_set = set(detected_sensors)
        expected_set = set(expected_sensors)
        true_positives = detected_set & expected_set
        false_negatives = expected_set - detected_set
        false_positives = detected_set - expected_set
        
        expected_count = len(expected_set)
        true_positive_count = len(true_positives)
        detected_count = len(detected_set)
        
        expected_sum += expected_count
        true_positive_sum += true_positive_count
        detected_sum += detected_count
        
        miss_ratio = len(false_negatives) / expected_count if expected_count > 0 else 0
        false_ratio = len(false_positives) / detected_count if detected_count > 0 else 0
        
        print(f"时段 {start_idx}-{end_idx}:")
        print(f"  期望异常传感器: {sorted(expected_set)}")
        print(f"  检测到异常传感器: {sorted(detected_set)}")
        print(f"  命中: {sorted(true_positives)}")
        print(f"  漏检: {sorted(false_negatives)}")
        print(f"  误检: {sorted(false_positives)}")
        print(f"  正确检测数: {true_positive_count}")
        print(f"  漏检比例: {miss_ratio:.2f}")
        print(f"  误检比例: {false_ratio:.2f}")
        print("-"*50)
    
    # 计算当前文件的召回率和精确率
    recall = (true_positive_sum / expected_sum) if expected_sum > 0 else 0
    precision = (true_positive_sum / detected_sum) if detected_sum > 0 else 0
    
    print("当前文件统计:")
    print(f"  总期望异常传感器数: {expected_sum}")
    print(f"  总正确检测数: {true_positive_sum}")
    print(f"  总检测到异常传感器数: {detected_sum}")
    print(f"  召回率: {recall*100:.1f}%")
    print(f"  精确率: {precision*100:.1f}%")
    print("="*60)
    
    return {"expected_sum": expected_sum,
            "true_positive_sum": true_positive_sum,
            "detected_sum": detected_sum}

if __name__ == "__main__":
    # 指定包含 omi- 开头 txt 文件的文件夹（仅处理 omi- 文件）
    label_folder = r"data\interpretation_label"
    txt_files = [f for f in os.listdir(label_folder)
                 if f.startswith("machine-") and f.endswith(".txt")]
    
    global_expected = 0
    global_true_positive = 0
    global_detected = 0
    
    for txt_filename in sorted(txt_files):
        prefix = txt_filename.split(".")[0]  # 例如 "omi-1"
        test_data_path = os.path.join("data/processed", f"{prefix}_test.pkl")
        test_labels_path = os.path.join("data/processed", f"{prefix}_test_label.pkl")
        
        if not (os.path.exists(test_data_path) and os.path.exists(test_labels_path)):
            print(f"缺少对应的测试数据或标签文件：{prefix}")
            continue
        
        test_data = pd.read_pickle(test_data_path)
        test_labels = pd.read_pickle(test_labels_path)
        if isinstance(test_data, np.ndarray):
            test_data = pd.DataFrame(test_data)
        if isinstance(test_labels, np.ndarray):
            test_labels = pd.DataFrame(test_labels, columns=["label"])
        elif isinstance(test_labels, pd.Series):
            test_labels = test_labels.to_frame(name="label")
        
        file_path = os.path.join(label_folder, txt_filename)
        file_stats = process_omi_file(file_path, test_data, test_labels)
        
        global_expected += file_stats["expected_sum"]
        global_true_positive += file_stats["true_positive_sum"]
        global_detected += file_stats["detected_sum"]
        
        # 输出当前文件后的全局统计
        current_recall = (global_true_positive / global_expected) if global_expected > 0 else 0
        current_precision = (global_true_positive / global_detected) if global_detected > 0 else 0
        print("当前累计总体统计:")
        print(f"  总期望异常传感器数: {global_expected}")
        print(f"  总正确检测数: {global_true_positive}")
        print(f"  总检测到异常传感器数: {global_detected}")
        print(f"  累计召回率: {current_recall*100:.1f}%")
        print(f"  累计精确率: {current_precision*100:.1f}%")
        print("="*60)
    
    # 最后输出全局总体统计
    global_recall = (global_true_positive / global_expected) if global_expected > 0 else 0
    global_precision = (global_true_positive / global_detected) if global_detected > 0 else 0
    
    print("最终总体统计:")
    print(f"  总期望异常传感器数: {global_expected}")
    print(f"  总正确检测数: {global_true_positive}")
    print(f"  总检测到异常传感器数: {global_detected}")
    print(f"  全局召回率: {global_recall*100:.1f}%")
    print(f"  全局精确率: {global_precision*100:.1f}%")



Processing file: data\interpretation_label\machine-1-1.txt
时段 15849-16395:
  期望异常传感器: [1, 9, 10, 12, 13, 14, 15]
  检测到异常传感器: [0, 1, 2, 3, 5, 6, 8, 9, 10, 11, 12, 13, 14, 15, 18, 19, 20, 21, 22, 23, 24, 25, 27, 28, 29, 30, 31, 32, 33, 34, 35]
  命中: [1, 9, 10, 12, 13, 14, 15]
  漏检: []
  误检: [0, 2, 3, 5, 6, 8, 11, 18, 19, 20, 21, 22, 23, 24, 25, 27, 28, 29, 30, 31, 32, 33, 34, 35]
  正确检测数: 7
  漏检比例: 0.00
  误检比例: 0.77
--------------------------------------------------
时段 16963-17517:
  期望异常传感器: [1, 2, 3, 4, 6, 7, 9, 10, 11, 12, 13, 14, 15, 16, 19, 20, 21, 22, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36]
  检测到异常传感器: [0, 1, 2, 3, 5, 6, 8, 9, 10, 11, 12, 13, 14, 15, 18, 19, 20, 21, 22, 23, 24, 25, 27, 28, 29, 30, 31, 32, 33, 34, 35]
  命中: [1, 2, 3, 6, 9, 10, 11, 12, 13, 14, 15, 19, 20, 21, 22, 24, 25, 27, 28, 29, 30, 31, 32, 33, 34, 35]
  漏检: [4, 7, 16, 26, 36]
  误检: [0, 5, 8, 18, 23]
  正确检测数: 26
  漏检比例: 0.16
  误检比例: 0.16
--------------------------------------------------
时段 18071-1852